Imports needed to merge and create the single dataframe.

In [182]:
import pandas as pd
import numpy as np
import json
import ast
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import scipy.stats as stats

Creating and importing the dataframes to be analyzed and merged.

In [183]:
movies_data=pd.read_csv('../Data/movies_metadata.csv')
ratings=pd.read_csv('../Data/ratings_small.csv')
ratings_full=pd.read_csv('../Data/ratings.csv')
links=pd.read_csv('../Data/links_small.csv')
links_full=pd.read_csv('../Data/links.csv')
keywords_to_add=pd.read_csv('../Data/keywords.csv')
credits=pd.read_csv('../Data/credits.csv')

Checking the datatypes for the credits data frame to show the actors and the crew information per movie.

In [184]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
cast    45476 non-null object
crew    45476 non-null object
id      45476 non-null int64
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


Since the cast and crew infromation are in a single string object per line, the as type library will be used to return all of these into lists of dictionaries that can be iterated through to extrat information. This will be done using a function that can wbe applied to the columns.

In [185]:
type(ast.literal_eval(credits['cast'][0]))

list

In [186]:
def string_to_list(row):
    row=ast.literal_eval(row)
    return row

In [187]:
credits['cast']=credits['cast'].apply(string_to_list)
credits['crew']=credits['crew'].apply(string_to_list)

This is the current check of the keys of the columns that will be extracted in order to obtain the cast and crew info. This will be used to pull the cast/crew member name, the name of thcharacter, the gender of the cast or crew member, the department of the crew member and the job of the crew member.

In [188]:
credits['cast'][0][0].keys()

dict_keys(['cast_id', 'character', 'credit_id', 'gender', 'id', 'name', 'order', 'profile_path'])

In [189]:
credits['crew'][0][0].keys()

dict_keys(['credit_id', 'department', 'gender', 'id', 'job', 'name', 'profile_path'])

In [190]:
credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': 'Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [191]:
credits['cast'][0]

[{'cast_id': 14,
  'character': 'Woody (voice)',
  'credit_id': '52fe4284c3a36847f8024f95',
  'gender': 2,
  'id': 31,
  'name': 'Tom Hanks',
  'order': 0,
  'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'},
 {'cast_id': 15,
  'character': 'Buzz Lightyear (voice)',
  'credit_id': '52fe4284c3a36847f8024f99',
  'gender': 2,
  'id': 12898,
  'name': 'Tim Allen',
  'order': 1,
  'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'},
 {'cast_id': 16,
  'character': 'Mr. Potato Head (voice)',
  'credit_id': '52fe4284c3a36847f8024f9d',
  'gender': 2,
  'id': 7167,
  'name': 'Don Rickles',
  'order': 2,
  'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'},
 {'cast_id': 17,
  'character': 'Slinky Dog (voice)',
  'credit_id': '52fe4284c3a36847f8024fa1',
  'gender': 2,
  'id': 12899,
  'name': 'Jim Varney',
  'order': 3,
  'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'},
 {'cast_id': 18,
  'character': 'Rex (voice)',
  'credit_id': '52fe4284c3a36847f8024fa5',
  'gender': 2,
  'id': 12900,
 

In [192]:
credits['crew'][0]

[{'credit_id': '52fe4284c3a36847f8024f49',
  'department': 'Directing',
  'gender': 2,
  'id': 7879,
  'job': 'Director',
  'name': 'John Lasseter',
  'profile_path': '/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg'},
 {'credit_id': '52fe4284c3a36847f8024f4f',
  'department': 'Writing',
  'gender': 2,
  'id': 12891,
  'job': 'Screenplay',
  'name': 'Joss Whedon',
  'profile_path': '/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg'},
 {'credit_id': '52fe4284c3a36847f8024f55',
  'department': 'Writing',
  'gender': 2,
  'id': 7,
  'job': 'Screenplay',
  'name': 'Andrew Stanton',
  'profile_path': '/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg'},
 {'credit_id': '52fe4284c3a36847f8024f5b',
  'department': 'Writing',
  'gender': 2,
  'id': 12892,
  'job': 'Screenplay',
  'name': 'Joel Cohen',
  'profile_path': '/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg'},
 {'credit_id': '52fe4284c3a36847f8024f61',
  'department': 'Writing',
  'gender': 0,
  'id': 12893,
  'job': 'Screenplay',
  'name': 'Alec Sokolow',
  'profile_path': '/v79vlRYi94BZUQnkkyzn

The below for loop is created in order to extract the actor's name, character name, actor gender and the associated movie id into a single row. This information will be created into a single dataframe for the castings that then can be grouped into casting counts.

In [193]:
cast_list=[]
for movie, cast in credits.set_index('id')['cast'].items():
    for cast_member in cast:
        credit={}
        credit['actor']=cast_member['name']
        credit['character']=cast_member['character']
        credit['gender']=cast_member['gender']
        credit['movie_id']=movie
        cast_list.append(credit)      
print(cast_list[:20])

[{'actor': 'Tom Hanks', 'character': 'Woody (voice)', 'gender': 2, 'movie_id': 862}, {'actor': 'Tim Allen', 'character': 'Buzz Lightyear (voice)', 'gender': 2, 'movie_id': 862}, {'actor': 'Don Rickles', 'character': 'Mr. Potato Head (voice)', 'gender': 2, 'movie_id': 862}, {'actor': 'Jim Varney', 'character': 'Slinky Dog (voice)', 'gender': 2, 'movie_id': 862}, {'actor': 'Wallace Shawn', 'character': 'Rex (voice)', 'gender': 2, 'movie_id': 862}, {'actor': 'John Ratzenberger', 'character': 'Hamm (voice)', 'gender': 2, 'movie_id': 862}, {'actor': 'Annie Potts', 'character': 'Bo Peep (voice)', 'gender': 1, 'movie_id': 862}, {'actor': 'John Morris', 'character': 'Andy (voice)', 'gender': 0, 'movie_id': 862}, {'actor': 'Erik von Detten', 'character': 'Sid (voice)', 'gender': 2, 'movie_id': 862}, {'actor': 'Laurie Metcalf', 'character': 'Mrs. Davis (voice)', 'gender': 1, 'movie_id': 862}, {'actor': 'R. Lee Ermey', 'character': 'Sergeant (voice)', 'gender': 2, 'movie_id': 862}, {'actor': 'Sar

In [194]:
castings=pd.DataFrame(cast_list)
print(castings.shape)
castings.sample(30)

(562474, 4)


,actor,character,gender,movie_id
417452,Master Ramesh,Chinna Kaaka Muttai,2,264038
428913,Joan Hackett,Lauriette,1,63438
551935,Emilio Linder,Brian,0,28445
351519,Jane Darwell,Mrs. Flaherty,1,82239
404651,Andrew Simpson,Saul,2,134693
123872,Toshirō Mifune,Rokurota Makabe,2,1059
322770,Björn Thors,Hannes,2,127913
418016,Bartosz Obuchowicz,Leon Pajak,2,207909
161672,Steven Anthony Lawrence,Mark Avery,2,9981
526509,Marcel Musters,Older policemen,0,223551


This dataframe is used to create a count of the number of times an actor has appeared in the dataset and it can be merged onto the casting dataframe. These counts will be used to establish the amount of work or how prolific an actor is in the dataset. An actor like Tom Hanks, or Samuel L Jackson will have severl credits while an actor like Jason Howard will have very few.

In [195]:
total_actor_count=castings.groupby('actor',as_index=False).count().drop(columns=['gender','movie_id']).rename(columns={'character':'total_actor_credits'})
total_actor_count.sample(20)

,actor,total_actor_credits
82080,James Komack,2
71036,Hannah Gunzig,1
152797,Peter Simonischek,5
12510,Annemarie Lawless,3
152793,Peter Simlinger,1
150329,Paul Rogers,9
8713,Anatoly Duatlov,1
110982,Laura Marinoni,1
77583,Ingrid van Bergen,7
89658,Jim Knaub,1


This data frame will establish a the total number of cast members that a movie has employed. This will show the number of cast members and will be able to establish how many actors are necessary per film.

In [196]:
total_cast_count=castings.groupby('movie_id',as_index=False).count().drop(columns=['character','gender']).rename(columns={'actor':'total_cast_per_movie'})
total_cast_count.sample(20)

,movie_id,total_cast_per_movie
13697,32689,6
10251,23945,12
27056,85927,5
30463,119965,7
36667,248543,15
491,634,23
862,1358,4
31425,131457,5
32937,157690,10
8770,19418,9


This will show how many times a character has appeared in the data set. This is important to show how prolific a character is in the dataset and if a particular character has appeared in the dataset and if this character may be a draw.

In [197]:
character_casting_count=castings.groupby('character',as_index=False).count().drop(columns=['gender','movie_id']).rename(columns={'actor':'character_per_role'}).sort_values('character_per_role',ascending=False)
character_casting_count.sample(10)

,character,character_per_role
61685,Dadan Karambolo,1
119278,Heibei,1
55091,Colonel Ricardo Diaz,1
258713,Savage (uncredited),1
311938,Young Shana,1
204456,Mr. Mohan,3
191405,Me Clermont,1
270842,Smithsonian Guard,1
267153,Shower Head,1
77605,Dr. Haseem,1


This is a count of the number of different genders in a movie and can show how diverse the cast is of a film in terms of gender.

In [198]:
movie_casting_count_by_gender=castings.groupby(['movie_id','gender'],as_index=False).count().drop(columns='character').rename(columns={'actor':'total_casting_count_by_gender'})
movie_casting_count_by_gender.sample(10)

,movie_id,gender,total_casting_count_by_gender
50809,47182,0,1
78615,115199,1,1
77876,111396,1,3
80311,123092,0,1
7555,7555,0,8
55636,54109,2,8
48912,44793,1,1
44031,40932,0,8
2800,1580,1,2
78608,115162,1,2


The count of the number of times an actor has appear, the number of times a character has appeared, the gender diversity of a film and the total number of cast members are merged onto a single dataframe. After looking into the data the character count and the gender counts do not have any more explanitory power, while the total film casting number and the amount of castings an actor has is.

In [199]:
castings=castings.merge(total_cast_count,on='movie_id')
castings=castings.merge(total_actor_count,on='actor')
castings=castings.merge(character_casting_count,on='character')
castings=castings.merge(movie_casting_count_by_gender,on=['movie_id','gender'])
castings.sample(20)

,actor,character,gender,movie_id,total_cast_per_movie,total_actor_credits,character_per_role,total_casting_count_by_gender
237059,Eric Dane,Doyle,2,430598,13,8,23,5
439970,Tom Bell,Henry Harding MP,2,20684,12,24,1,7
436129,George Jonson,Blondie,0,15728,10,1,15,4
180207,Jim Carrey,Grinch,2,8871,17,42,1,8
144009,Theodore Rand,Dance Specialty,0,27638,65,1,4,36
66547,Haruma Miura,Noto,0,36172,8,10,1,3
117935,Melissa Temme,Maestra Maggie,0,211954,49,1,1,36
5458,Mario Andretti,Himself,2,23638,7,5,11437,6
125037,David Doyle,Mel,2,36850,17,14,46,12
137744,Franciska Spahic,Joanne,0,40461,12,1,39,12


The below dictionary shows the amount of the work an actor will need to be have a certain level of work. The most working actors will have 75% of the casting count, the second most will have 50% of the work, the second least will have 25% and the least will have below 25%. This will allow the grouping of how valuable an actor is and will create different columns to show this information.

In [200]:
catings_A_to_D_list=dict(castings['total_actor_credits'].describe()[4:7])
catings_A_to_D_list

{'25%': 2.0, '50%': 6.0, '75%': 17.0}

In [201]:
castings['most_used_actor']=castings['total_actor_credits'].apply(lambda x: 1 if x>=catings_A_to_D_list['75%'] else 0)
castings.sample(10)

,actor,character,gender,movie_id,total_cast_per_movie,total_actor_credits,character_per_role,total_casting_count_by_gender,most_used_actor
448200,Daniel Auteuil,Marquis de Sade,2,72356,6,38,4,3,1
526350,Ikue Ōtani,Pikachu (voice),1,50087,13,17,4,5,1
512232,Julian Schaffner,Andrew Davis,0,335676,5,1,2,4,0
481351,House Peters Jr.,Steve - Border Inspection Officer,2,47149,17,5,1,8,0
230338,Justin Chatwin,Nick Powell,2,9785,20,13,2,11,0
344551,David Eigenberg,Steve Brady,2,37786,32,8,2,9,0
268651,Connie Jackson,Nurse,0,276908,9,1,499,5,0
256777,Clayton Big Eagle,Pawnee #2,0,581,57,1,1,38,0
221945,Judy Greer,Laura,1,55632,11,48,297,7,1
265773,Christopher Knight,TV Anchorman,2,13552,14,3,5,7,0


In [202]:
castings['second_most_used_actor']=castings['total_actor_credits'].apply(lambda x: 1 if (x<catings_A_to_D_list['75%']) and (x>=catings_A_to_D_list['50%'])  else 0)
castings.sample(10)

,actor,character,gender,movie_id,total_cast_per_movie,total_actor_credits,character_per_role,total_casting_count_by_gender,most_used_actor,second_most_used_actor
33514,Keisuke Horibe,,0,45441,19,15,25782,10,0,1
6865,Will Arnett,Himself,2,191024,8,40,11437,7,1,0
539915,Son Ye-jin,Baek Jang-mi,1,24037,5,11,1,1,0,1
306347,Victor von Schirach,Cashier,0,56596,17,1,67,10,0,0
420309,Kosuke Toyohara,Yamagishi,0,282069,16,8,1,6,0,1
551586,Kauko Kokkonen,vääpeli Pippurinen,0,148435,8,2,1,6,0,0
38449,Miho Ninagawa,Momoka,0,20431,32,1,1,20,0,0
493238,Mary Wickes,Mamie,1,218299,18,28,19,2,1,0
160960,James Duval,Rick,2,116983,6,29,125,4,1,0
333830,Javier Botet,Pepo,0,282983,8,15,2,6,0,1


In [203]:
castings['second_least_used_actor']=castings['total_actor_credits'].apply(lambda x: 1 if (x<catings_A_to_D_list['50%']) and (x>=catings_A_to_D_list['25%'])  else 0)
castings.sample(10)

,actor,character,gender,movie_id,total_cast_per_movie,total_actor_credits,character_per_role,total_casting_count_by_gender,most_used_actor,second_most_used_actor,second_least_used_actor
81527,Oliver Hudson,Kyle,2,187596,20,7,121,12,0,1,0
62385,Matthew Lintz,,2,74018,7,4,25782,3,0,0,1
15959,Jimmy Kimmel,Himself,0,26765,40,14,11437,1,0,1,0
73653,Jim Malinda,Pusher,0,88390,21,2,4,9,0,0,1
134311,Val Kilmer,Angel,2,51250,7,70,105,5,1,0,0
396780,Keith Hampshire,Other Computers,0,21356,19,3,1,12,0,0,1
29159,Aurora Quattrocchi,Tenutaria bordello,0,10867,40,7,1,21,0,1,0
316083,Glória Pires,Helena,0,21752,12,5,75,4,0,0,1
252067,Jamie Farr,Mario,0,43710,9,13,122,1,0,1,0
362515,Edouard Bamporiki,Emmanuel,2,81215,4,1,9,1,0,0,0


In [204]:
castings['least_used_actor']=castings['total_actor_credits'].apply(lambda x: 1 if (x<catings_A_to_D_list['25%']) else 0)
castings.sample(10)

,actor,character,gender,movie_id,total_cast_per_movie,total_actor_credits,character_per_role,total_casting_count_by_gender,most_used_actor,second_most_used_actor,second_least_used_actor,least_used_actor
469449,Paprika Steen,Ursula Østdal,0,50656,14,19,1,6,1,0,0,0
166339,Richard D'Alessandro,Abbie Hoffman,0,13,66,1,4,24,0,0,0,1
510389,Lucy Davis,Director #2,1,79519,11,12,2,4,0,1,0,0
355214,Lizzy Caplan,Agent Lacey,1,228967,38,19,1,4,1,0,0,0
271913,Bobby Calloway,Bored Student (uncredited),0,356189,28,1,1,21,0,0,0,1
314,Steven Gary Banks,Billy The Mime,2,15258,105,1,1,64,0,0,0,1
429969,Jean Eustache,Friendly Man,2,11222,20,2,2,4,0,0,1,0
227992,Tom Wilkinson,Tuppy,2,15184,8,67,1,4,1,0,0,0
18657,Liam Price,Billy,0,59266,22,1,255,9,0,0,0,1
234981,Robert Axelrod,Racially Confused Veteran,2,79896,34,6,1,12,0,1,0,0


This information is showing the total number of actors is in each actor category. This will be used to determine how valuable and actor is and how the distribution is in relation to the total amount of cast members.

In [205]:
castings_per_movie=castings.groupby('movie_id',as_index=False).sum().drop(columns=['gender','total_actor_credits','character_per_role','total_casting_count_by_gender','total_cast_per_movie']).rename(columns={'most_used_actor':'most_used_actor_per_movie','second_most_used_actor':'second_most_used_actor_per_movie','second_least_used_actor':'second_least_used_actor_per_movie','least_used_actor':'least_used_actor_per_movie'})
castings_per_movie.sample(10)

,movie_id,most_used_actor_per_movie,second_most_used_actor_per_movie,second_least_used_actor_per_movie,least_used_actor_per_movie
37290,259894,0,1,5,2
19113,47383,0,1,2,7
20895,53914,4,3,2,4
7020,15338,2,3,4,0
4200,10534,11,1,3,0
41935,391088,5,10,31,31
15807,39045,1,2,0,1
18566,45665,1,0,0,0
38678,292834,7,4,1,0
30913,125217,3,12,17,6


In [206]:
castings=castings.merge(castings_per_movie,on='movie_id')
castings.sample(10)

,actor,character,gender,movie_id,total_cast_per_movie,total_actor_credits,character_per_role,total_casting_count_by_gender,most_used_actor,second_most_used_actor,second_least_used_actor,least_used_actor,most_used_actor_per_movie,second_most_used_actor_per_movie,second_least_used_actor_per_movie,least_used_actor_per_movie
263192,Pawel Iwanicki,Abramow,0,31021,6,1,1,4,0,0,0,1,1,1,3,1
133257,Darlene Tompkins,Patsy,1,18228,16,2,23,8,0,0,1,0,8,2,5,1
204925,Belinda Owino,Gemma,0,273248,18,2,22,5,0,0,1,0,11,4,3,0
443460,Haley Barnes,Production Crew Member 2,0,137669,58,1,1,46,0,0,0,1,0,0,12,46
261586,Piero Natoli,Michele,0,12308,23,2,32,10,0,0,1,0,5,6,9,3
278239,Stephen Root,Gordon,2,9472,19,60,79,17,1,0,0,0,14,2,2,1
454092,Barbro Hiort af Ornäs,Stig Helmer's mother,1,17172,19,7,1,2,0,1,0,0,0,7,6,6
405562,Nathalie Besançon,La ministre,0,16277,19,3,1,11,0,0,1,0,2,2,8,7
181726,Ray Liotta,Lance Wescott,2,22327,16,69,1,9,1,0,0,0,10,3,2,1
535331,Sanja Mikitišin,Jasnina majka,0,95775,10,1,1,8,0,0,0,1,0,0,2,8


All unnecssary information is dropped and all that is left is the number of people in a casting group and how large the total cast is. All duplicates are dropped in order insure that there is one to one data for movies.

In [207]:
castings.drop(columns=['actor','character','gender','total_actor_credits','character_per_role','total_casting_count_by_gender','most_used_actor','second_most_used_actor','second_least_used_actor','least_used_actor'],inplace=True)
castings.sample(10)

,movie_id,total_cast_per_movie,most_used_actor_per_movie,second_most_used_actor_per_movie,second_least_used_actor_per_movie,least_used_actor_per_movie
255039,3074,31,19,8,1,3
146504,343018,52,2,0,10,40
393170,170315,14,7,4,2,1
319345,273856,5,3,0,0,2
462049,68761,19,5,5,4,5
388157,110624,13,4,6,2,1
175627,141679,30,2,8,10,10
214785,270654,34,7,3,12,12
406483,25364,32,18,7,2,5
242564,44909,19,4,5,7,3


In [208]:
castings.drop_duplicates(inplace=True)
castings.sample(10)

,movie_id,total_cast_per_movie,most_used_actor_per_movie,second_most_used_actor_per_movie,second_least_used_actor_per_movie,least_used_actor_per_movie
480916,9086,18,14,3,1,0
561197,158942,3,0,1,0,2
241871,258435,7,5,2,0,0
558616,25869,6,5,1,0,0
208318,257214,10,3,3,4,0
86479,101733,9,0,2,4,3
60421,96243,15,1,7,5,2
517214,95410,7,2,4,0,1
345856,394244,3,0,1,1,1
547536,256679,1,0,0,1,0


The below check and for loop shows how the crew information is broken up. This shows where the crew department, crew job crew member name and crew member gender is contained. The below for loop breaks this information a way so that this can be imported into a dataframe for analysis.

In [209]:
credits['crew'][0][0]

{'credit_id': '52fe4284c3a36847f8024f49',
 'department': 'Directing',
 'gender': 2,
 'id': 7879,
 'job': 'Director',
 'name': 'John Lasseter',
 'profile_path': '/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg'}

In [210]:
crew_list=[]
for movie, crew in credits.set_index('id')['crew'].items():
    for crew_member in crew:
        credit={}
        credit['crew_member']=crew_member['name']
        credit['department']=crew_member['department']
        credit['job']=crew_member['job']
        credit['gender']=crew_member['gender']
        credit['movie_id']=movie
        crew_list.append(credit)

In [211]:
crews=pd.DataFrame(crew_list)
print(crews.shape)
crews.sample(30)

(464314, 5)


,crew_member,department,job,gender,movie_id
303611,Ian Fenton,Visual Effects,Visual Effects Supervisor,0,265016
170895,Haruhide Ishiguro,Camera,Director of Photography,0,14945
292673,Derek Lee,Writing,Screenplay,0,210947
2081,J. Peter Robinson,Sound,Music,2,12158
163015,Brian Koppelman,Production,Producer,2,1491
431456,Amanda Laws,Editing,Editor,0,376534
18356,Jean Giraud,Art,Conceptual Design,2,348
104662,Roger Page,Art,Leadman,0,9562
410056,Harvey Kahn,Production,Executive Producer,2,278774
44824,Gregg Barbanell,Sound,Foley,2,10349


This dataframe is used to create a count of the number of times an crew member has appeared in the dataset and it can be merged onto the crew dataframe. These counts will be used to establish the amount of work or how prolific a crew member is in the dataset. An crew member like Sofia Coppola will have several credits while an actor like Zoe White may have very few.

In [212]:
crew_member_count=crews.groupby('crew_member', as_index=False).count().sort_values(by=['department'],ascending=False).rename(columns={'department':'crew_member_count'}).drop(columns=['job','gender','movie_id'])
crew_member_count.sample(10)

,crew_member,crew_member_count
37243,Dmitriy Kirillov,1
126492,Robert J. Avrech,5
67937,Jemma Skidmore,1
133828,Sean Bernard,1
32782,David Carter,2
32387,Dave Steinbach,1
134452,Seong-jae Kim,1
72714,John Dines,1
141655,Susan Reilly LeHane,3
19961,C. Roy Hunter,3


This dataframe shows the total number of crew members that are in the dataset per movie.

In [213]:
movie_crew_count=crews.groupby(['movie_id'],as_index=False).count().drop(columns=['department','job','gender']).sort_values(by='crew_member',ascending=False).rename(columns={'crew_member':'total_movie_crew'})
movie_crew_count.sample(10)

,movie_id,total_movie_crew
28010,90147,8
43903,407799,2
35359,189875,1
36487,213842,1
34076,163312,3
16711,41326,4
42157,348537,4
12476,29144,2
32153,132144,6
39479,282035,119


This shows how big a certain department is in per film.

In [214]:
movie_crew_department_count=crews.groupby(['movie_id','department'],as_index=False).count().drop(columns=['job','gender']).rename(columns={'crew_member':'total_crew_per_department'}).sort_values('total_crew_per_department',ascending=False)
movie_crew_department_count.sample(10)

,movie_id,department,total_crew_per_department
69340,26865,Writing,2
103338,46500,Production,2
113785,55604,Production,1
171398,210302,Writing,1
169325,199420,Lighting,2
155994,133469,Directing,1
104357,47139,Production,3
95693,42532,Production,4
143155,98369,Crew,1
179894,259387,Writing,2


This shows how many people are employed in each crew job.

In [215]:
movie_department_jobs_count=crews.groupby(['movie_id','job'],as_index=False).count().drop(columns=['department','gender']).rename(columns={'crew_member':'total_crew_job_count'}).sort_values('total_crew_job_count',ascending=False)
movie_department_jobs_count.sample(10)

,movie_id,job,total_crew_job_count
65792,9804,Costume Supervisor,1
50875,8292,Steadicam Operator,1
284517,210577,Makeup Effects,1
129541,24149,Director,1
190409,48660,Director,2
227125,79221,Co-Writer,1
53988,8869,Makeup Effects,1
220333,72881,Costume Design,1
24129,1640,Associate Producer,1
114636,18166,Producer,1


This shows the gender diversity of each movie's crew.

In [216]:
gender_count_of_cast=crews.groupby(['movie_id','gender'],as_index=False).count().drop(columns=['department','job']).rename(columns={'crew_member':'count_of_gender_per_movie'}).sort_values('count_of_gender_per_movie',ascending=False)
gender_count_of_cast.sample(10)

,movie_id,gender,count_of_gender_per_movie
65036,149206,1,1
66340,162145,2,1
72802,249457,0,1
49935,71370,0,11
54261,84306,2,8
9330,9877,0,14
16350,14672,0,10
48760,67377,1,1
36526,42464,2,6
4066,2652,2,9


The below data frame merges the number of credits a crew member has in the dataset, the total crew in a movie, the total crew members in the film department of the movie, the total number of people are employed in an individual job and the gender diveristy. After review, the department counts, job counts and gender diversity hold little explaintory power.

In [217]:
crews=crews.merge(movie_crew_count,on='movie_id')
crews=crews.merge(crew_member_count,on='crew_member')
crews=crews.merge(movie_department_jobs_count,on=['job','movie_id'])
crews=crews.merge(gender_count_of_cast,on=['movie_id','gender'])
crews=crews.merge(movie_crew_department_count,on=['movie_id','department'])
crews.head(20)

,crew_member,department,job,gender,movie_id,total_movie_crew,crew_member_count,total_crew_job_count,count_of_gender_per_movie,total_crew_per_department
0,John Lasseter,Directing,Director,2,862,106,85,1,29,3
1,Roman Figun,Directing,Layout,0,862,106,2,2,73,3
2,Desirée Mourad,Directing,Layout,0,862,106,1,2,73,3
3,John Lasseter,Writing,Original Story,2,862,106,85,4,29,8
4,Andrew Stanton,Writing,Original Story,2,862,106,33,4,29,8
5,Pete Docter,Writing,Original Story,2,862,106,18,4,29,8
6,Joss Whedon,Writing,Screenplay,2,862,106,24,4,29,8
7,Andrew Stanton,Writing,Screenplay,2,862,106,33,4,29,8
8,Joel Cohen,Writing,Screenplay,2,862,106,7,4,29,8
9,Joe Ranft,Writing,Original Story,0,862,106,14,4,73,8


The below the information shows a dictionary that breaks the crew counts of the crew members, or how many credits a crew member has in total. The 75% mark will be broken into the most working crew member. The second most working will be between 50% and 75% of the crew member credit counts. The second least will be between 25% and 50% of the credit counts of the crew member counts. The least used crew member are used below 25% of the time.

In [218]:
crew_count_dict=dict(crews['crew_member_count'].describe()[4:7])
crew_count_dict

{'25%': 2.0, '50%': 5.0, '75%': 14.0}

In [219]:
crews['most_used_crew_member']=crews['crew_member_count'].apply(lambda x: 1 if x >= crew_count_dict['75%'] else 0)
crews.sample(10)

,crew_member,department,job,gender,movie_id,total_movie_crew,crew_member_count,total_crew_job_count,count_of_gender_per_movie,total_crew_per_department,most_used_crew_member
391088,Adam Mason,Editing,Editor,2,10041,16,18,2,6,2,1
393775,Hiroshi Yamaguchi,Writing,Screenplay,0,50388,4,1,2,4,3,0
72233,Sarawut Chincharoen,Art,Leadman,0,10204,135,1,1,109,20,0
433665,Steffen Haars,Writing,Writer,2,79723,4,6,2,2,2,0
220490,Johnny Harris,Sound,Original Music Composer,0,44698,13,4,1,8,1,0
267970,Kenji Kawai,Sound,Original Music Composer,2,9323,31,25,1,12,2,1
407334,Jon Shenk,Crew,Cinematography,0,376228,8,6,1,8,1,0
106819,Christine Bell,Writing,Novel,0,63020,19,1,1,5,2,0
13588,Bill Taliaferro,Art,Set Designer,0,95,108,5,9,70,24,0
406730,Johan Söderberg,Editing,Editor,0,121740,12,4,1,11,1,0


In [220]:
crews['second_most_used_crew_member']=crews['crew_member_count'].apply(lambda x: 1 if (x >= crew_count_dict['50%']) and (x < crew_count_dict['75%']) else 0)
crews.sample(10)

,crew_member,department,job,gender,movie_id,total_movie_crew,crew_member_count,total_crew_job_count,count_of_gender_per_movie,total_crew_per_department,most_used_crew_member,second_most_used_crew_member
284750,Richard Day,Art,Art Direction,2,105548,15,101,1,7,2,1,0
350792,Jen Santucci,Crew,Property Master,0,287636,17,1,1,12,2,0,0
370957,Aben Kandel,Writing,Story,2,110336,11,14,1,6,2,1,0
122090,Sukh Gill,Visual Effects,Visual Effects Coordinator,0,70044,17,2,1,11,4,0,0
248092,Nobutaka Yoshino,Art,Art Direction,0,88529,9,1,1,5,1,0,0
225969,Valentin Mircea,Camera,Camera Operator,0,297291,52,4,1,33,6,0,0
324763,Mark Christopher,Directing,Director,2,106222,1,5,1,1,1,0,1
359903,George Provis,Art,Art Direction,2,55152,11,9,1,7,1,0,1
421038,Liza Antonova,Production,Producer,0,352720,13,1,7,12,7,0,0
319854,Mario Montanari,Production,Producer,0,117890,10,2,2,6,2,0,0


In [221]:
crews['second_least_used_crew_member']=crews['crew_member_count'].apply(lambda x: 1 if (x >= crew_count_dict['25%']) and (x < crew_count_dict['50%']) else 0)
crews.sample(10)

,crew_member,department,job,gender,movie_id,total_movie_crew,crew_member_count,total_crew_job_count,count_of_gender_per_movie,total_crew_per_department,most_used_crew_member,second_most_used_crew_member,second_least_used_crew_member
148764,Joe Simon,Writing,Characters,2,271110,119,4,2,28,5,0,0,1
64204,Derek Brechin,Editing,Editor,2,14636,65,12,1,10,4,0,1,0
329736,Patrick Wachsberger,Production,Producer,2,85446,11,14,4,7,6,1,0,0
123934,Ken Davis,Art,Production Design,0,96,26,1,1,8,4,0,0,0
337905,Peter Marley,Art,Art Direction,0,283691,36,1,4,28,5,0,0,0
85007,Suzanne Kelly,Costume & Make-Up,Set Costumer,1,56651,27,2,1,5,2,0,0,1
249222,Allison Meadows,Production,Casting,0,67748,20,3,1,13,5,0,0,1
271645,Alfio Contini,Camera,Director of Photography,2,194485,12,17,1,4,1,1,0,0
263918,Branimir Babic,Art,Art Direction,0,9956,19,4,1,10,2,0,0,1
258628,Edward Carrere,Art,Art Direction,2,49616,9,18,1,6,2,1,0,0


In [222]:
crews['least_used_crew_member']=crews['crew_member_count'].apply(lambda x: 1 if x < crew_count_dict['25%']  else 0)
crews.sample(10)

,crew_member,department,job,gender,movie_id,total_movie_crew,crew_member_count,total_crew_job_count,count_of_gender_per_movie,total_crew_per_department,most_used_crew_member,second_most_used_crew_member,second_least_used_crew_member,least_used_crew_member
68356,Bradley Woodbridge,Art,Painter,0,7364,107,1,1,75,11,0,0,0,1
66261,Mariana Mihailova,Costume & Make-Up,Makeup Artist,0,207933,108,1,11,82,33,0,0,0,1
290198,Walter M. Simonds,Art,Art Direction,0,15919,22,13,2,7,3,0,1,0,0
157007,Robert T. Striem,Production,Location Manager,0,16558,52,3,1,34,5,0,0,1,0
186549,Robert L. Richards,Writing,Screenplay,2,14551,26,8,2,14,3,0,1,0,0
322539,Alexandre Desplat,Sound,Original Music Composer,2,6173,8,69,1,5,1,1,0,0,0
414617,Vicki Vandegrift,Sound,Foley,0,290729,32,17,1,19,7,1,0,0,0
437969,Cassie Miggins,Art,Production Design,0,83330,8,1,1,3,2,0,0,0,1
391362,Manu Chao,Sound,Original Music Composer,0,1127,9,1,2,3,2,0,0,0,1
153024,Frank Kowalski,Writing,Story,0,11942,47,1,2,33,4,0,0,0,1


The below data frame is grouped by in order to show the total number of casting in each group per movie and will provide the total groupings. This information is then merged onto the crews dataframe.

In [223]:
crews_proflic_count_per_movie=crews.groupby('movie_id',as_index=False).sum().drop(columns=['gender','total_movie_crew','crew_member_count','total_crew_job_count','count_of_gender_per_movie','total_crew_per_department']).rename(columns={'most_used_crew_member':'most_used_crew_member_per_movie','second_most_used_crew_member':'second_most_used_crew_member_per_movie','second_least_used_crew_member':'second_least_used_crew_member_per_movie','least_used_crew_member':'least_used_crew_member_per_movie'})
crews_proflic_count_per_movie.sample(10)


,movie_id,most_used_crew_member_per_movie,second_most_used_crew_member_per_movie,second_least_used_crew_member_per_movie,least_used_crew_member_per_movie
44297,425498,1,0,2,1
13462,31894,0,2,3,1
15206,37429,0,0,0,1
23057,62522,0,0,2,0
21285,54580,1,8,4,0
7640,16538,17,29,21,15
7218,15698,7,3,2,1
8841,19486,0,2,0,0
3624,9788,10,18,4,3
15048,36996,0,3,9,16


In [224]:
crews=crews.merge(crews_proflic_count_per_movie,on='movie_id')
crews.sample(10)

,crew_member,department,job,gender,movie_id,total_movie_crew,crew_member_count,total_crew_job_count,count_of_gender_per_movie,total_crew_per_department,most_used_crew_member,second_most_used_crew_member,second_least_used_crew_member,least_used_crew_member,most_used_crew_member_per_movie,second_most_used_crew_member_per_movie,second_least_used_crew_member_per_movie,least_used_crew_member_per_movie
117541,Stephen Diener,Production,Executive Producer,2,10014,30,4,2,13,6,0,0,1,0,10,4,9,7
419142,Jean-Paul Rouve,Writing,Writer,0,139725,3,3,2,3,2,0,0,1,0,0,1,2,0
107533,Andrés Levin,Sound,Original Music Composer,0,30319,21,3,1,13,1,0,0,1,0,3,5,10,3
231561,Peter Williams,Art,Set Decoration,0,107327,16,2,1,9,2,0,0,1,0,4,3,4,5
163748,Dara Weintraub,Production,Co-Producer,0,4723,36,6,1,18,22,0,1,0,0,8,16,8,4
91125,Daniela Kurrie,Costume & Make-Up,Tailor,0,109439,92,1,1,60,16,0,0,0,1,32,31,19,10
442083,Matt Lang,Camera,Camera Operator,0,239895,27,10,3,26,3,0,1,0,0,0,6,10,11
410891,Mike Wiser,Writing,Writer,0,273334,3,1,2,3,2,0,0,0,1,0,0,2,1
305864,William Frye,Production,Producer,0,7227,13,5,1,5,3,0,1,0,0,3,7,1,2
355363,Jenna Ricker,Writing,Writer,0,343010,18,4,2,16,4,0,0,1,0,1,4,10,3


The unneccessary information is dropped from the dataframe and then duplicates are removed in order to merge onto a single dataframe in the single movies dataframe.

In [225]:
crews.drop(columns=['crew_member','department','job','gender','crew_member_count','total_crew_job_count','count_of_gender_per_movie','total_crew_per_department','most_used_crew_member','second_most_used_crew_member','second_least_used_crew_member','least_used_crew_member'],inplace=True)
crews.sample(10)


,movie_id,total_movie_crew,most_used_crew_member_per_movie,second_most_used_crew_member_per_movie,second_least_used_crew_member_per_movie,least_used_crew_member_per_movie
367680,241843,49,7,20,13,9
120313,139933,52,5,11,23,13
299450,77650,9,6,2,1,0
16146,14161,115,28,48,28,11
111998,9879,28,13,11,3,1
118030,23514,47,22,13,9,3
218842,157825,5,2,1,2,0
10712,11970,49,21,12,10,6
279335,36610,50,1,8,17,24
165438,9072,36,15,15,3,3


crews.drop_duplicates(inplace=True)
crews.sample(10)

The ratings and ratings full dataframes are checked. Since the ratings dataframe contains no new information it is dropped.

In [226]:
ratings_full.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [227]:
ratings_full.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

In [228]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [229]:
ratings.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

In [230]:
print(ratings_full.shape)
print(ratings.shape)

(26024289, 4)
(100004, 4)


In [231]:
ratings_full.columns

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')

The below groupbys show the largest user rating that a film got, the lowest rating a film got, the average rating a film got, the median rating a film got and the a mode rating that a film got.

In [232]:
max_ratings_per_movie=ratings_full.groupby(['movieId'],as_index=False).max().sort_values(['rating'],ascending=False).drop(columns=['userId','timestamp']).rename(columns={'rating':'max_rating'})
max_ratings_per_movie.sample(10)

,movieId,max_rating
518,522,5.0
34854,148160,2.5
19564,96706,4.0
7892,8574,5.0
44648,175125,3.0
4519,4613,5.0
28994,131202,2.5
39474,161708,1.0
2857,2942,5.0
1252,1280,5.0


In [233]:
min_ratings_per_movie=ratings_full.groupby(['movieId'],as_index=False).min().sort_values(['rating'],ascending=False).drop(columns=['userId','timestamp']).rename(columns={'rating':'min_rating'})
min_ratings_per_movie.sample(10)

,movieId,min_rating
16673,84150,3.0
30348,135180,0.5
17738,88973,2.5
125,127,0.5
35339,149568,0.5
14653,73125,3.0
36550,153210,3.0
20132,98847,1.0
32045,140135,2.0
14391,71940,2.5


In [234]:
total_users_per_movie=ratings_full.groupby(['movieId'],as_index=False).count().sort_values(['rating'],ascending=False).drop(columns=['rating','timestamp']).rename(columns={'userId':'total_rating_per_movie'})
total_users_per_movie.sample(10)

,movieId,total_rating_per_movie
34151,146044,4
4342,4436,184
38023,157464,1
2483,2567,4483
2854,2939,542
9173,27015,1
18864,93621,3
24402,114569,2
13247,64548,6
34085,145831,1


In [235]:
median_ratings=ratings_full.groupby(['movieId'],as_index=False).median().drop(columns=['userId','timestamp']).rename(columns={'rating':'median_rating'})
median_ratings.sample(10)

,movieId,median_rating
35578,150280,3.5
5004,5099,3.5
37889,157116,4.5
44732,175359,3.5
24641,115607,0.5
27659,126428,3.5
15186,77266,3.5
22221,106452,4.0
38043,157565,5.0
41855,168156,1.5


In [236]:
average_ratings=ratings_full.groupby(['movieId'],as_index=False).mean().drop(columns=['userId','timestamp']).rename(columns={'rating':'average_rating'})
average_ratings.head(10)

,movieId,average_rating
0,1,3.888157
1,2,3.236953
2,3,3.175550
3,4,2.875713
4,5,3.079565
5,6,3.841764
6,7,3.372105
7,8,3.129191
8,9,3.008365
9,10,3.431841


In [237]:
mode_ratings=ratings_full.groupby(['movieId'],as_index=False).agg(lambda x: int(stats.mode(x)[0])).drop(columns=['userId','timestamp']).rename(columns={'rating':'mode_rating'})
mode_ratings.head(10)

,movieId,mode_rating
0,1,4.0
1,2,3.0
2,3,3.0
3,4,3.0
4,5,3.0
5,6,4.0
6,7,3.0
7,8,3.0
8,9,3.0
9,10,3.0


The keywords dataframe shows the search terms in a dataframe. The keywords are stored as a string object so the conversion function is applied. Then it is iterated through in an order to break each keyword in its own dataframe to have further analysis done on it.

In [238]:
keywords_to_add.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [239]:
keywords_to_add['keywords']=keywords_to_add['keywords'].apply(string_to_list)

In [240]:
keywords_to_add.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [241]:
keywords_to_add.keywords[2]

[{'id': 1495, 'name': 'fishing'},
 {'id': 12392, 'name': 'best friend'},
 {'id': 179431, 'name': 'duringcreditsstinger'},
 {'id': 208510, 'name': 'old men'}]

In [242]:
keywords_list=[]
for movie, keywords in keywords_to_add.set_index('id')['keywords'].items():
    for keyword in keywords:
        keys={}
        keys['keywords_one']=keyword['name']
        keys['movie_id']=movie
        keywords_list.append(keys)

In [243]:
key_words_to_manipulate=pd.DataFrame(keywords_list)
key_words_to_manipulate.head(30)

,keywords_one,movie_id
0,jealousy,862
1,toy,862
2,boy,862
3,friendship,862
4,friends,862
5,rivalry,862
6,boy next door,862
7,new toy,862
8,toy comes to life,862
9,board game,8844


The below corpus is created to have senitment analysis done on the search terms. This will show the overall sentiment that is associated witb the words to show how positive, or negative the films search terms are. This is run through then appended onto a single dataframe.

In [244]:
keywords_corpus=list(key_words_to_manipulate['keywords_one'])
keywords_corpus[:10]

['jealousy',
 'toy',
 'boy',
 'friendship',
 'friends',
 'rivalry',
 'boy next door',
 'new toy',
 'toy comes to life',
 'board game']

In [245]:
sia=SentimentIntensityAnalyzer()

In [246]:
all_scores_keywords=[]
for i in keywords_corpus:
    score=sia.polarity_scores(i)
    score['keyword']=i
    all_scores_keywords.append(score)
df_keyword_sentiments=pd.DataFrame(all_scores_keywords)
df_keyword_sentiments.head(10)

,neg,neu,pos,compound,keyword
0,1.0,0.0,0.0,-0.3182,jealousy
1,0.0,1.0,0.0,0.0000,toy
2,0.0,1.0,0.0,0.0000,boy
3,0.0,0.0,1.0,0.4404,friendship
4,0.0,0.0,1.0,0.4767,friends
5,0.0,1.0,0.0,0.0000,rivalry
6,0.0,1.0,0.0,0.0000,boy next door
7,0.0,1.0,0.0,0.0000,new toy
8,0.0,1.0,0.0,0.0000,toy comes to life
9,0.0,1.0,0.0,0.0000,board game


This groupby shows the usage of a single dataframe.

In [247]:
keyword_count=key_words_to_manipulate.groupby(['keywords_one'],as_index=False).count().rename(columns={'movie_id':'key_word_total_appearances'})
keyword_count.sample(30)

,keywords_one,key_word_total_appearances
652,anti authority,2
1301,bareback rider,1
13301,pizza delivery,6
242,ailul al aswad,1
926,assassin,126
4300,cyanide,1
14333,rasputin,4
8813,incomprehensible plot,1
1956,bodyguard,34
6905,franz liszt,1


The below shows the number of keywords that are associated with a single movie.

In [248]:
movie_keyword_count=key_words_to_manipulate.groupby(['movie_id'],as_index=False).count().rename(columns={'keywords_one':'total_movie_keywords'})
movie_keyword_count.sample(30)

,movie_id,total_movie_keywords
9114,23045,1
15285,44222,1
20997,87368,1
29454,345888,1
7986,18979,3
6157,14047,1
22236,105989,8
14576,42585,3
20654,84628,4
15619,45622,5


The below dataframe merges the senitment of each keyword, how often the keyword appears in the dataframe, and how many keywords a film is using.

In [249]:
key_words_to_manipulate=key_words_to_manipulate.merge(keyword_count,on=['keywords_one'])
key_words_to_manipulate=key_words_to_manipulate.merge(movie_keyword_count,on=['movie_id'])
key_words_to_manipulate=key_words_to_manipulate.merge(df_keyword_sentiments,left_on=['keywords_one'],right_on=['keyword'])
key_words_to_manipulate.drop(columns=['keyword'],inplace=True)
key_words_to_manipulate.sample(30)

,keywords_one,movie_id,key_word_total_appearances,total_movie_keywords,neg,neu,pos,compound
10832118,woman director,129363,3115,1,0.0,1.0,0.0,0.0000
304299,prostitute,31007,234,6,0.0,1.0,0.0,0.0000
16162763,suspense,2654,590,18,0.0,1.0,0.0,0.0000
21514775,teacher,38724,214,4,0.0,1.0,0.0,0.0000
3734087,independent film,28855,1930,8,0.0,1.0,0.0,0.0000
17466212,murder,227668,1308,3,1.0,0.0,0.0,-0.6908
21618662,teenager,43727,441,12,0.0,1.0,0.0,0.0000
29485100,wild west,79192,25,7,0.0,1.0,0.0,0.0000
1746598,independent film,61689,1930,15,0.0,1.0,0.0,0.0000
3987376,independent film,57248,1930,1,0.0,1.0,0.0,0.0000


The below the information shows a dictionary that breaks the keyword counts of the keyword appearances, or how many appearance a keywords has in total. The 75% mark will be broken into the most used keyword. The second most used will be between 50% and 75% of the keyword usage counts. The second least will be between 25% and 50% of the usage of keyword counts. The least used keyword are used below 25% of the time.

In [250]:
key_words_class_dict=dict(key_words_to_manipulate.key_word_total_appearances.describe(percentiles=[.25,.5,.75])[4:7])
key_words_class_dict

{'25%': 268.0, '50%': 1308.0, '75%': 3115.0}

In [251]:
key_words_to_manipulate['most_used_keywords']=key_words_to_manipulate['key_word_total_appearances'].apply(lambda x: 1 if x>=key_words_class_dict['75%'] else 0)
key_words_to_manipulate.sample(20)

,keywords_one,movie_id,key_word_total_appearances,total_movie_keywords,neg,neu,pos,compound,most_used_keywords
7138545,woman director,8665,3115,6,0.0,1.000,0.000,0.0000,1
6674633,woman director,3050,3115,2,0.0,1.000,0.000,0.0000,1
15952966,homosexuality,81707,175,3,0.0,1.000,0.000,0.0000,0
11201321,woman director,248808,3115,3,0.0,1.000,0.000,0.0000,1
27608784,musical,10615,734,5,0.0,1.000,0.000,0.0000,0
18570790,murder,64624,1308,22,1.0,0.000,0.000,-0.6908,0
4917799,woman director,359870,3115,6,0.0,1.000,0.000,0.0000,1
19358716,london england,382591,238,11,0.0,1.000,0.000,0.0000,0
29230485,lesbian relationship,361069,65,12,0.0,1.000,0.000,0.0000,0
336257,independent film,5879,1930,14,0.0,1.000,0.000,0.0000,0


In [252]:
key_words_to_manipulate['second_most_used_keywords']=key_words_to_manipulate['key_word_total_appearances'].apply(lambda x: 1 if x>=key_words_class_dict['50%'] and x<key_words_class_dict['75%'] else 0)
key_words_to_manipulate.sample(20)

,keywords_one,movie_id,key_word_total_appearances,total_movie_keywords,neg,neu,pos,compound,most_used_keywords,second_most_used_keywords
2083485,independent film,48862,1930,7,0.0,1.000,0.000,0.0000,0,1
17881108,murder,16077,1308,4,1.0,0.000,0.000,-0.6908,0,1
2854972,independent film,36762,1930,4,0.0,1.000,0.000,0.0000,0,1
20373125,sport,20443,544,1,0.0,1.000,0.000,0.0000,0,0
19883849,biography,501,629,16,0.0,1.000,0.000,0.0000,0,0
6584637,woman director,124843,3115,1,0.0,1.000,0.000,0.0000,1,0
23114497,escape,46525,218,6,0.0,0.000,1.000,0.1779,0,0
28387585,greece,23049,31,6,0.0,1.000,0.000,0.0000,0,0
4820355,woman director,10464,3115,7,0.0,1.000,0.000,0.0000,1,0
22454911,based on novel,305943,835,5,0.0,0.465,0.535,0.3182,0,0


In [253]:
key_words_to_manipulate['second_least_used_keywords']=key_words_to_manipulate['key_word_total_appearances'].apply(lambda x: 1 if x>=key_words_class_dict['25%'] and x<key_words_class_dict['50%'] else 0)
key_words_to_manipulate.sample(20)

,keywords_one,movie_id,key_word_total_appearances,total_movie_keywords,neg,neu,pos,compound,most_used_keywords,second_most_used_keywords,second_least_used_keywords
23836709,father daughter relationship,32023,111,16,0.000,1.000,0.0,0.0000,0,0,0
19621088,female nudity,36808,563,7,0.000,1.000,0.0,0.0000,0,0,1
25080312,sequel,215302,439,3,0.000,1.000,0.0,0.0000,0,0,1
1708467,independent film,38753,1930,6,0.000,1.000,0.0,0.0000,0,1,0
17359352,murder,41090,1308,12,1.000,0.000,0.0,-0.6908,0,1,0
29251844,talent-scout,168903,3,14,0.000,1.000,0.0,0.0000,0,0,0
16331599,suspense,12633,590,7,0.000,1.000,0.0,0.0000,0,0,1
3215378,independent film,35689,1930,1,0.000,1.000,0.0,0.0000,0,1,0
5030514,woman director,62564,3115,2,0.000,1.000,0.0,0.0000,1,0,0
13429237,woman director,324794,3115,4,0.000,1.000,0.0,0.0000,1,0,0


In [254]:
key_words_to_manipulate['least_used_keywords']=key_words_to_manipulate['key_word_total_appearances'].apply(lambda x: 1 if x<=key_words_class_dict['25%'] else 0)
key_words_to_manipulate.sample(20)

,keywords_one,movie_id,key_word_total_appearances,total_movie_keywords,neg,neu,pos,compound,most_used_keywords,second_most_used_keywords,second_least_used_keywords,least_used_keywords
7932486,woman director,16201,3115,3,0.0,1.000,0.000,0.0000,1,0,0,0
6329245,woman director,15256,3115,5,0.0,1.000,0.000,0.0000,1,0,0,0
6952387,woman director,77434,3115,1,0.0,1.000,0.000,0.0000,1,0,0,0
4169865,marriage,15919,258,9,0.0,1.000,0.000,0.0000,0,0,0,1
3976267,independent film,20860,1930,1,0.0,1.000,0.000,0.0000,0,1,0,0
25032801,sequel,52017,439,7,0.0,1.000,0.000,0.0000,0,0,1,0
23675557,gang,138832,149,6,0.0,1.000,0.000,0.0000,0,0,0,1
19705758,female nudity,42499,563,9,0.0,1.000,0.000,0.0000,0,0,1,0
585389,independent film,41966,1930,15,0.0,1.000,0.000,0.0000,0,1,0,0
6729606,woman director,9489,3115,3,0.0,1.000,0.000,0.0000,1,0,0,0


The below groupby shows the average keyword sentiment per movie. This shows the overall sentiment of the SEO of the film.

In [255]:
average_keywords_sentiments_by_movie=key_words_to_manipulate.groupby('movie_id',as_index=False).mean().drop(columns=['key_word_total_appearances','total_movie_keywords','most_used_keywords','second_most_used_keywords','second_least_used_keywords','least_used_keywords']).rename(columns={'neg':'average_keyword_negative_sentiment_per_movie','neu':'average_keyword_neutral_sentiment_per_movie','pos':'average_keyword_postive_sentiment_per_movie','compound':'average_keyword_compound_sentiment_per_movie'})
average_keywords_sentiments_by_movie.sample(10)

,movie_id,average_keyword_negative_sentiment_per_movie,average_keyword_neutral_sentiment_per_movie,average_keyword_postive_sentiment_per_movie,average_keyword_compound_sentiment_per_movie
21202,90094,0.000000,1.000000,0.000000,0.000000
25435,189696,0.118000,0.882000,0.000000,-0.088856
14400,42146,0.080488,0.919512,0.000000,-0.027366
6491,14742,0.000000,1.000000,0.000000,0.000000
13706,39450,0.000000,1.000000,0.000000,0.000000
13158,37606,0.000000,1.000000,0.000000,0.000000
6468,14683,0.341463,0.652919,0.005618,-0.209864
14974,43462,0.000000,1.000000,0.000000,0.000000
3880,10193,0.000000,0.803646,0.196354,0.035572
3432,9629,0.754032,0.245968,0.000000,-0.395555


The below dataframe shows the below count of the number of keywords per cateogry.

In [256]:
key_word_counts_by_movie=key_words_to_manipulate.groupby('movie_id',as_index=False).sum().drop(columns=['key_word_total_appearances','total_movie_keywords','neg','neu','pos','compound']).rename(columns={'most_used_keywords':'most_used_keywords_per_movie','second_most_used_keywords':'second_most_used_keywords_per_movie','second_least_used_keywords':'second_least_used_keywords_per_movie','least_used_keywords':'least_used_keywords_per_movie'})
key_word_counts_by_movie.sample(10)

,movie_id,most_used_keywords_per_movie,second_most_used_keywords_per_movie,second_least_used_keywords_per_movie,least_used_keywords_per_movie
10486,27561,0,0,636,108
12064,33065,0,0,734,286
29819,362150,0,0,626,178
23075,121598,3115,0,0,0
18664,65573,0,0,0,5
11215,29979,0,0,293,136
22716,114444,0,0,0,11
12855,36630,0,0,1895,1071
12280,33924,0,0,0,207
25851,204994,0,0,0,118


The below dataframe merges all dataframes into a single unit. All unneccessary columns and duplicates are dropped and then the single dataframe will be kept in order to keep the information into a single location.

In [257]:
key_words_to_manipulate=key_words_to_manipulate.merge(key_word_counts_by_movie, on='movie_id')
key_words_to_manipulate=key_words_to_manipulate.merge(average_keywords_sentiments_by_movie, on='movie_id')
key_words_to_manipulate.sample(10)

,keywords_one,movie_id,key_word_total_appearances,total_movie_keywords,neg,neu,pos,compound,most_used_keywords,second_most_used_keywords,second_least_used_keywords,least_used_keywords,most_used_keywords_per_movie,second_most_used_keywords_per_movie,second_least_used_keywords_per_movie,least_used_keywords_per_movie,average_keyword_negative_sentiment_per_movie,average_keyword_neutral_sentiment_per_movie,average_keyword_postive_sentiment_per_movie,average_keyword_compound_sentiment_per_movie
28700052,television,9965,54,3,0.0,1.000,0.000,0.0000,0,0,0,1,0,0,0,62,0.000000,1.000000,0.000000,0.000000
16553260,woman director,114189,3115,1,0.0,1.000,0.000,0.0000,1,0,0,0,3115,0,0,0,0.000000,1.000000,0.000000,0.000000
16232841,woman director,91266,3115,1,0.0,1.000,0.000,0.0000,1,0,0,0,3115,0,0,0,0.000000,1.000000,0.000000,0.000000
23709834,murder,59189,1308,7,1.0,0.000,0.000,-0.6908,0,1,0,0,0,1308,0,370,0.779499,0.220501,0.000000,-0.538478
2524572,independent film,28784,1930,8,0.0,1.000,0.000,0.0000,0,1,0,0,0,3238,1289,307,0.270583,0.637004,0.092413,-0.131955
5378615,independent film,20348,1930,12,0.0,1.000,0.000,0.0000,0,1,0,0,0,1930,0,301,0.000000,0.998655,0.001345,0.000788
9243344,based on novel,5719,835,6,0.0,0.465,0.535,0.3182,0,0,1,0,0,1308,2156,112,0.389821,0.485256,0.124923,-0.192129
7993196,woman director,242224,3115,15,0.0,1.000,0.000,0.0000,1,0,0,0,3115,0,285,418,0.005987,0.993921,0.000092,-0.003342
5059024,independent film,4927,1930,9,0.0,1.000,0.000,0.0000,0,1,0,0,0,1930,0,1011,0.000000,0.990289,0.009711,0.004913
6800004,independent film,48373,1930,1,0.0,1.000,0.000,0.0000,0,1,0,0,0,1930,0,0,0.000000,1.000000,0.000000,0.000000


In [258]:
key_words_to_manipulate.drop(columns=['keywords_one','key_word_total_appearances','neg','neu','pos','compound','most_used_keywords','second_most_used_keywords','second_least_used_keywords','least_used_keywords'],inplace=True)

In [259]:
key_words_to_manipulate.drop_duplicates(inplace=True)
key_words_to_manipulate.sample(10)

,movie_id,total_movie_keywords,most_used_keywords_per_movie,second_most_used_keywords_per_movie,second_least_used_keywords_per_movie,least_used_keywords_per_movie,average_keyword_negative_sentiment_per_movie,average_keyword_neutral_sentiment_per_movie,average_keyword_postive_sentiment_per_movie,average_keyword_compound_sentiment_per_movie
25812576,80389,4,0,0,351,213,0.645745,0.342199,0.012057,-0.331306
29595532,107076,1,0,0,0,14,0.796000,0.204000,0.000000,-0.599400
29028548,375170,1,0,0,0,196,1.000000,0.000000,0.000000,-0.318200
9143190,12435,4,3115,0,0,76,0.002194,0.997806,0.000000,-0.001006
29652121,214464,1,0,0,0,245,0.000000,1.000000,0.000000,0.000000
29774666,215656,1,0,0,0,2,0.000000,1.000000,0.000000,0.000000
21733173,37923,6,0,1308,0,451,0.883457,0.116543,0.000000,-0.558005
28825107,2335,1,0,0,0,11,0.000000,1.000000,0.000000,0.000000
29769783,17481,4,0,0,0,22,0.000000,0.123364,0.876636,0.313918
28811051,72711,8,0,0,0,165,0.004715,0.358921,0.636364,0.344495


The links information is disregarded in due to no useful information being contained.

In [260]:
links.head() #disregarded due to no new info

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [261]:
links_full.head() #disregarded due to no new info

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [262]:
print(links_full.shape)
print(links.shape)

(45843, 3)
(9125, 3)


The movies metadata is checked for all information that can be used and for null. Due to using the popularity information, the data will only use films that are released and do not have nulls in the following categories: release dates, runtimes and IMBD data.

In [263]:
movies_data.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [264]:
movies_data.shape

(45466, 24)

In [265]:
movies_data.isna().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

In [266]:
movies_data['status'].value_counts()

Released           45014
Rumored              230
Post Production       98
In Production         20
Planned               15
Canceled               2
Name: status, dtype: int64

In [267]:
movies_data=movies_data[movies_data['status']=='Released'] #other rows dropped due to model needing to predict revenue of already released films

In [268]:
movies_data.isna().sum()

adult                        0
belongs_to_collection    40548
budget                       0
genres                       0
homepage                 37308
id                           0
imdb_id                     15
original_language           10
original_title               0
overview                   920
popularity                   0
poster_path                373
production_companies         0
production_countries         0
release_date                78
revenue                      0
runtime                    251
spoken_languages             0
status                       0
tagline                  24714
title                        0
video                        0
vote_average                 0
vote_count                   0
dtype: int64

In [269]:
movies_data=movies_data[movies_data.release_date.notna()]

In [270]:
movies_data.isna().sum()

adult                        0
belongs_to_collection    40470
budget                       0
genres                       0
homepage                 37241
id                           0
imdb_id                     12
original_language           10
original_title               0
overview                   907
popularity                   0
poster_path                332
production_companies         0
production_countries         0
release_date                 0
revenue                      0
runtime                    240
spoken_languages             0
status                       0
tagline                  24649
title                        0
video                        0
vote_average                 0
vote_count                   0
dtype: int64

In [271]:
movies_data=movies_data[movies_data.runtime.notna()]

In [272]:
movies_data.isna().sum()

adult                        0
belongs_to_collection    40237
budget                       0
genres                       0
homepage                 37001
id                           0
imdb_id                     10
original_language           10
original_title               0
overview                   667
popularity                   0
poster_path                315
production_companies         0
production_countries         0
release_date                 0
revenue                      0
runtime                      0
spoken_languages             0
status                       0
tagline                  24409
title                        0
video                        0
vote_average                 0
vote_count                   0
dtype: int64

In [273]:
movies_data=movies_data[movies_data.imdb_id.notna()]

In [274]:
movies_data.isna().sum()

adult                        0
belongs_to_collection    40229
budget                       0
genres                       0
homepage                 36991
id                           0
imdb_id                      0
original_language           10
original_title               0
overview                   667
popularity                   0
poster_path                314
production_companies         0
production_countries         0
release_date                 0
revenue                      0
runtime                      0
spoken_languages             0
status                       0
tagline                  24401
title                        0
video                        0
vote_average                 0
vote_count                   0
dtype: int64

In [275]:
movies_data.shape

(44686, 24)

The release date is being converted into a date time datatype and then broken into separate year month and day columns.

In [276]:
movies_data['release_date']=pd.to_datetime(movies_data.release_date)

In [277]:
movies_data['release_year']=movies_data['release_date'].dt.year

In [278]:
movies_data['release_month']=movies_data['release_date'].dt.month

In [279]:
movies_data['release_day']=movies_data['release_date'].dt.day

The below information is created into booleans to show if this information is present in the dataset and then the obejects are dropped.

In [280]:
movies_data['has_homepage']=movies_data['homepage'].notna().astype(int)
movies_data['is_in_series']=movies_data['belongs_to_collection'].notna().astype(int)
movies_data['has_tagline']=movies_data['tagline'].notna().astype(int)
movies_data['has_poster']=movies_data['poster_path'].notna().astype(int)
movies_data['video']=movies_data['video'].astype(int)
movies_data['has_overview']=movies_data['overview'].notna().astype(int)

In [281]:
movies_data[['homepage','has_homepage']].sample(10) #will drop homepage due to info able to info being encoded elsewhere

,homepage,has_homepage
35741,NaN,0
10572,http://www.firstdescentmovie.com/,1
6212,NaN,0
902,NaN,0
16930,NaN,0
43201,NaN,0
22242,NaN,0
39455,NaN,0
27678,NaN,0
27096,NaN,0


In [282]:
movies_data[['has_tagline','tagline']].sample(10)#will drop tagline due to info able to info being encoded elsewhere

,has_tagline,tagline
27512,0,NaN
39279,0,NaN
11191,1,When every college turned them down. . . they ...
8531,0,NaN
2120,1,Trust everyone... But always cut the cards.
5119,0,NaN
16367,0,NaN
3365,1,Invasion from Outer Space!...Sights too weird ...
42880,0,NaN
23563,0,NaN


In [283]:
movies_data[['belongs_to_collection','is_in_series']].sample(10) #will drop collection due to info able to info being encoded elsewhere

,belongs_to_collection,is_in_series
19534,NaN,0
40779,NaN,0
237,NaN,0
42103,NaN,0
38494,NaN,0
4404,"{'id': 101471, 'name': 'Return of the Living D...",1
8243,NaN,0
24704,NaN,0
19656,NaN,0
38444,NaN,0


In [284]:
movies_data[['has_poster','poster_path']].sample(10) #will drop poster due to info able to info being encoded elsewhere

,has_poster,poster_path
30181,1,/m0Our7wh91YieK86piDmmiOWkeR.jpg
18950,1,/pozxglZvlZWIuhoyNMRSHUFUzmf.jpg
19020,1,/cilpU63HE0nsbqMytvO3Mlltotw.jpg
21116,1,/jn8L1QdWWX5c0NUOLjzaSXtZrbt.jpg
4008,1,/tZlFs9GY6iM8xsze5b0PYHG9hPX.jpg
19776,1,/mpBFF9Jwsn9eFXLb7wULAXxl8pH.jpg
7812,1,/kdvYnPBI9stvBaaNzRumr8gbIXH.jpg
14747,1,/jeLZ9RM8WbKmAkOWXBTN42gBBtU.jpg
7076,1,/iplKGhtKCoA7SZHCnm3H5Grf2l4.jpg
29562,1,/35Ic3juHhF1RMFsyrC2kQQM1Ab3.jpg


The adult, budget and revenue information is dropped due to not having useful information.

In [285]:
movies_data['adult'].value_counts(normalize=True) #will drop due to few films being in this column

False    0.999821
True     0.000179
Name: adult, dtype: float64

In [286]:
movies_data.budget.value_counts(normalize=True) #drop budget due to 80% of the values being zero which is an impossible movie spend

0           0.801951
5000000     0.006400
10000000    0.005796
20000000    0.005438
2000000     0.005371
              ...   
1120175     0.000022
980385      0.000022
106000      0.000022
566000      0.000022
291000      0.000022
Name: budget, Length: 1215, dtype: float64

All of the genre information is created into its own boolean columns to show that the information is present and the film is associated in that genre.

In [287]:
print(movies_data['genres'].unique())

["[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"
 "[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]"
 "[{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]" ...
 "[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name': 'Comedy'}, {'id': 28, 'name': 'Action'}, {'id': 10751, 'name': 'Family'}]"
 "[{'id': 28, 'name': 'Action'}, {'id': 9648, 'name': 'Mystery'}, {'id': 53, 'name': 'Thriller'}, {'id': 27, 'name': 'Horror'}]"
 "[{'id': 10751, 'name': 'Family'}, {'id': 16, 'name': 'Animation'}, {'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]"]


In [288]:
movies_data['is_animation']=movies_data['genres'].str.contains('Animation').astype(int)
movies_data['is_comedy']=movies_data['genres'].str.contains('Comedy').astype(int)
movies_data['is_family']=movies_data['genres'].str.contains('Family').astype(int)
movies_data['is_adventure']=movies_data['genres'].str.contains('Adventure').astype(int)
movies_data['is_fantasy']=movies_data['genres'].str.contains('Fantasy').astype(int)
movies_data['is_romance']=movies_data['genres'].str.contains('Romance').astype(int)
movies_data['is_crime']=movies_data['genres'].str.contains('Crime').astype(int)
movies_data['is_thriller']=movies_data['genres'].str.contains('Thriller').astype(int)
movies_data['is_mystery']=movies_data['genres'].str.contains('Mystery').astype(int)
movies_data['is_science_fiction']=movies_data['genres'].str.contains('Science Fiction').astype(int)
movies_data['is_music']=movies_data['genres'].str.contains('Music').astype(int)
movies_data['is_horror']=movies_data['genres'].str.contains('Horror').astype(int)
movies_data['is_war']=movies_data['genres'].str.contains('War').astype(int)
movies_data['is_documentary']=movies_data['genres'].str.contains('Documentary').astype(int)
movies_data['is_foreign']=movies_data['genres'].str.contains('Foreign').astype(int)
movies_data['is_tv_movie']=movies_data['genres'].str.contains('TV Movie').astype(int)
movies_data['is_history']=movies_data['genres'].str.contains('History').astype(int)
movies_data['is_drama']=movies_data['genres'].str.contains('Drama').astype(int)
movies_data['is_western']=movies_data['genres'].str.contains('Western').astype(int)
movies_data['is_action']=movies_data['genres'].str.contains('Action').astype(int)

In [289]:
movies_data[(movies_data['is_animation']==1)|(movies_data['is_comedy']==1)|
                      (movies_data['is_history']==1)|(movies_data['is_tv_movie']==1)|
                     (movies_data['is_fantasy']==1)|(movies_data['is_romance']==1)|
                     (movies_data['is_crime']==1)&(movies_data['is_thriller']==1)|
                      (movies_data['is_mystery']==1)|(movies_data['is_family']==1)|
                     (movies_data['is_adventure']==1)|(movies_data['is_science_fiction']==1)|
                     (movies_data['is_music']==1)|(movies_data['is_horror']==1)|
                     (movies_data['is_war']==1)|(movies_data['is_documentary']==1)|
                     (movies_data['is_foreign']==1)|(movies_data['is_drama']==1)|
                     (movies_data['is_western']==1)|(movies_data['is_action']==1)].sample(20)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,is_music,is_horror,is_war,is_documentary,is_foreign,is_tv_movie,is_history,is_drama,is_western,is_action
40773,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}]",NaN,370203,tt4716560,sv,The Swedish Theory of Love,Internationally Sweden is seen as a perfect so...,...,0,0,0,1,0,0,0,0,0,0
36257,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,378200,tt4871980,en,The Perfect Match,"Terrence J. stars as Charlie, a playboy who's ...",...,0,0,0,0,0,0,0,0,0,0
35962,False,"{'id': 203029, 'name': 'Witchcraft Collection'...",0,"[{'id': 27, 'name': 'Horror'}]",NaN,46695,tt0245567,en,Witchcraft,A new mother and her child move into her mothe...,...,0,1,0,0,0,0,0,0,0,0
26405,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",NaN,70877,tt1640486,en,Inside Out,For the last thirteen years AJ (Triple H) has ...,...,0,0,0,0,0,0,0,1,0,0
35591,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,99729,tt0023878,en,Central Airport,Aviator Jim Blaine and his brother Neil are ri...,...,0,0,0,0,0,0,0,1,0,0
7399,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,5248,tt0164221,en,Virtual Sexuality,Desperate to find her perfect man Justine goes...,...,0,0,0,0,0,0,0,0,0,0
20060,False,NaN,42000000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",NaN,80278,tt1649419,en,Lo imposible,"In December 2004, close-knit family Maria, Hen...",...,0,0,0,0,0,0,0,1,0,0
25318,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 53, 'name...",NaN,136278,tt2078552,en,Blood,Thriller charting the moral collapse of a poli...,...,0,0,0,0,0,0,0,1,0,0
392,False,"{'id': 439409, 'name': 'Fear Collection', 'pos...",0,"[{'id': 27, 'name': 'Horror'}, {'id': 53, 'nam...",NaN,75555,tt0113043,en,The Fear,A weekend of 'fear therapy' turns into a night...,...,0,1,0,0,0,0,0,0,0,0
26694,False,NaN,12000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,226672,tt2369041,en,Legendary: Tomb of the Dragon,Travis and his team travel to China in search ...,...,0,0,0,0,0,0,0,0,0,1


The below value is the threshold how how many appearance a value should have in the dataset in order to have its own value. 1% will have enough explanitory power in order to determine value.

In [290]:
min_threshold=movies_data.shape[0]*.01
print(min_threshold)

446.86


The below information contains all of the production company information. This is iterated through a for loop into a new dataframe and then uses a groupby in order to determine how prolific a production company is. If the company has 100 or more credits, it is deemed valuable to the data. These companies are then iterated through the dataset and given there own unique boolean columns that allow the film to mark if the information is associated with the film.

In [291]:
movies_data['production_companies'].unique()

array(["[{'name': 'Pixar Animation Studios', 'id': 3}]",
       "[{'name': 'TriStar Pictures', 'id': 559}, {'name': 'Teitler Film', 'id': 2550}, {'name': 'Interscope Communications', 'id': 10201}]",
       "[{'name': 'Warner Bros.', 'id': 6194}, {'name': 'Lancaster Gate', 'id': 19464}]",
       ...,
       "[{'name': 'Westdeutscher Rundfunk (WDR)', 'id': 7025}, {'name': 'Working Title Films', 'id': 10163}, {'name': '20th Century Fox Television', 'id': 16323}, {'name': 'CanWest Global Communications', 'id': 38978}]",
       "[{'name': 'Sine Olivia', 'id': 19653}]",
       "[{'name': 'Yermoliev', 'id': 88753}]"], dtype=object)

In [292]:
movie_prod_company = []
for index, prod_companies in movies_data['production_companies'].iteritems():
    prod_companies = string_to_list(prod_companies)
    for each in prod_companies:
        movie_prod_company_dict = {}
        movie_prod_company_dict['movie_id'] = index
        movie_prod_company_dict['prod_comp'] = each['name']
        movie_prod_company.append(movie_prod_company_dict)

In [293]:
print(movie_prod_company_dict)

{'movie_id': 45464, 'prod_comp': 'Yermoliev'}


In [294]:
df_prod_cos = pd.DataFrame(data=movie_prod_company)
df_prod_cos.head()

,movie_id,prod_comp
0,0,Pixar Animation Studios
1,1,TriStar Pictures
2,1,Teitler Film
3,1,Interscope Communications
4,2,Warner Bros.


In [295]:
df_prod_cos = df_prod_cos.groupby('prod_comp').count().sort_values(by='movie_id', ascending=False)
df_prod_cos.sample(20)

,movie_id
prod_comp,
Demitri Samaha Productions,1
Achille Piazzi Produzioni Cinematografica,1
Mission Pictures,3
Korea Films,1
Timothy Burrill Productions,5
Cott Productions,2
Zéno Films,1
First Foot Films,2
Omega Entertainment,5


In [296]:
df_prod_cos['to_add']=(df_prod_cos['movie_id']>99).astype(int)
df_prod_cos=df_prod_cos[df_prod_cos['to_add']==1]
print(df_prod_cos.shape)
df_prod_cos.head(30)

(29, 2)


,movie_id,to_add
prod_comp,,
Warner Bros.,1248,1
Metro-Goldwyn-Mayer (MGM),1075,1
Paramount Pictures,997,1
Twentieth Century Fox Film Corporation,834,1
Universal Pictures,827,1
Columbia Pictures Corporation,447,1
Canal+,435,1
Columbia Pictures,429,1
RKO Radio Pictures,290,1


In [297]:
prod_cos_to_add=list(df_prod_cos.index)
print(prod_cos_to_add)
print(len(prod_cos_to_add))

['Warner Bros.', 'Metro-Goldwyn-Mayer (MGM)', 'Paramount Pictures', 'Twentieth Century Fox Film Corporation', 'Universal Pictures', 'Columbia Pictures Corporation', 'Canal+', 'Columbia Pictures', 'RKO Radio Pictures', 'New Line Cinema', 'United Artists', 'Walt Disney Pictures', 'Touchstone Pictures', 'TriStar Pictures', 'Mosfilm', 'Miramax Films', 'France 2 Cinéma', 'Centre National de la Cinématographie (CNC)', 'Toho Company', 'BBC Films', 'Gaumont', 'StudioCanal', 'Relativity Media', 'British Broadcasting Corporation (BBC)', 'Walt Disney Productions', 'TF1 Films Production', 'Orion Pictures', 'Village Roadshow Pictures', 'Regency Enterprises']
29


In [298]:
for i in prod_cos_to_add:
        movies_data[i]=movies_data['production_companies'].str.contains(i).astype(int)

In [299]:
print(movies_data['Metro-Goldwyn-Mayer (MGM)'].value_counts())
movies_data['Metro-Goldwyn-Mayer (MGM)']=movies_data['production_companies'].str.contains('Metro-Goldwyn-Mayer').astype(int)
print(movies_data['Metro-Goldwyn-Mayer (MGM)'].value_counts())
print(movies_data['Centre National de la Cinématographie (CNC)'].value_counts())
movies_data['Centre National de la Cinématographie (CNC)']=movies_data['production_companies'].str.contains('Centre National de la Cinématographie').astype(int)
print(movies_data['Centre National de la Cinématographie (CNC)'].value_counts())
print(movies_data['British Broadcasting Corporation (BBC)'].value_counts())
movies_data['British Broadcasting Corporation (BBC)']=movies_data['production_companies'].str.contains('British Broadcasting Corporation').astype(int)
print(movies_data['British Broadcasting Corporation (BBC)'].value_counts())

0    44686
Name: Metro-Goldwyn-Mayer (MGM), dtype: int64
0    43584
1     1102
Name: Metro-Goldwyn-Mayer (MGM), dtype: int64
0    44686
Name: Centre National de la Cinématographie (CNC), dtype: int64
0    44487
1      199
Name: Centre National de la Cinématographie (CNC), dtype: int64
0    44686
Name: British Broadcasting Corporation (BBC), dtype: int64
0    44568
1      118
Name: British Broadcasting Corporation (BBC), dtype: int64


The spoken language are iterated throguh a single list and then create a boolean column to ensure that the column if it has approximately 450 or more instances of the language being spoken.

In [300]:
movies_data['spoken_languages'].unique()

array(["[{'iso_639_1': 'en', 'name': 'English'}]",
       "[{'iso_639_1': 'en', 'name': 'English'}, {'iso_639_1': 'fr', 'name': 'Français'}]",
       "[{'iso_639_1': 'en', 'name': 'English'}, {'iso_639_1': 'es', 'name': 'Español'}]",
       ...,
       "[{'iso_639_1': 'sv', 'name': 'svenska'}, {'iso_639_1': 'de', 'name': 'Deutsch'}]",
       "[{'iso_639_1': 'ar', 'name': 'العربية'}, {'iso_639_1': 'pl', 'name': 'Polski'}]",
       "[{'iso_639_1': 'ff', 'name': 'Fulfulde'}, {'iso_639_1': 'en', 'name': 'English'}]"],
      dtype=object)

In [301]:
spoken_languages=movies_data.spoken_languages.apply(string_to_list)
spoken_laguages_unique=[]
for row in spoken_languages:
    for item in row:
        language=item['name']
        spoken_laguages_unique.append(language)
spoken_laguages_unique=list(set(spoken_laguages_unique))

In [302]:
print(len(spoken_laguages_unique))
print(spoken_laguages_unique[:10])
spoken_laguages_unique.remove('')
print(spoken_laguages_unique[:10])
print(len(spoken_laguages_unique))
spoken_laguages_unique.remove(r'?????')
print(spoken_laguages_unique[:10])
print(len(spoken_laguages_unique))
spoken_laguages_unique.remove(r'??????')
print(spoken_laguages_unique[:10])
print(len(spoken_laguages_unique))

75
['', 'ภาษาไทย', 'Bokmål', 'Deutsch', 'Dansk', '日本語', 'Español', 'Tiếng Việt', 'ਪੰਜਾਬੀ', 'Română']
['ภาษาไทย', 'Bokmål', 'Deutsch', 'Dansk', '日本語', 'Español', 'Tiếng Việt', 'ਪੰਜਾਬੀ', 'Română', 'Gaeilge']
74
['ภาษาไทย', 'Bokmål', 'Deutsch', 'Dansk', '日本語', 'Español', 'Tiếng Việt', 'ਪੰਜਾਬੀ', 'Română', 'Gaeilge']
73
['ภาษาไทย', 'Bokmål', 'Deutsch', 'Dansk', '日本語', 'Español', 'Tiếng Việt', 'ਪੰਜਾਬੀ', 'Română', 'Gaeilge']
72


In [303]:
for i in spoken_laguages_unique:
    if sum(movies_data['spoken_languages'].str.contains(i).astype(int))>min_threshold:
        movies_data[i]=movies_data['spoken_languages'].str.contains(i).astype(int)
    else: pass

The production company are iterated throguh a single list and then create a boolean column to ensure that the column if it has approximately 450 or more instances of the production company appearance.

In [304]:
production_countries=movies_data.production_countries.apply(string_to_list)
production_countries_unique=[]
for row in production_countries:
    for item in row:
        country=item['name']
        production_countries_unique.append(country)
production_countries_unique=list(set(production_countries_unique))
print(len(production_countries_unique))
production_countries_unique[:10]

160


['Uganda',
 'Yugoslavia',
 'Hong Kong',
 'Israel',
 'Mauritania',
 'Armenia',
 'Algeria',
 'Peru',
 'Aruba',
 'Burkina Faso']

In [305]:
for i in production_countries_unique:
    if sum(movies_data['production_countries'].str.contains(i).astype(int))>min_threshold:
        movies_data[i]=movies_data['production_countries'].str.contains(i).astype(int)
    else: pass

In [306]:
movies_data.sample(10)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,Japan,Spain,Canada,France,Russia,India,Australia,United Kingdom,South Korea,United States of America
6967,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,16814,tt0099316,en,Crazy People,A bitter ad executive who has reached his brea...,...,0,0,0,0,0,0,0,0,0,1
29553,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,204686,tt0051066,en,Ten Thousand Bedrooms,"In this musical-comedy, Dean Martin plays an A...",...,0,0,0,0,0,0,0,0,0,1
39742,False,NaN,0,"[{'id': 16, 'name': 'Animation'}, {'id': 10751...",http://veggietales.com/,275136,tt0297450,en,VeggieTales: The Ultimate Silly Song Countdown,The Pirates Who Don't Do Anything present the ...,...,0,0,0,0,0,0,0,0,0,1
38373,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}]",http://www.imdb.com/title/tt0380419/,173587,tt0380419,nl,Hollands Licht,There's an ancient myth that the light in Holl...,...,0,0,0,0,0,0,0,0,0,0
23380,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,94752,tt0069986,en,The Doll Squad,Squad of beautiful government agents tries to ...,...,0,0,0,0,0,0,0,0,0,0
1690,False,NaN,1000000,"[{'id': 53, 'name': 'Thriller'}, {'id': 80, 'n...",NaN,9558,tt0099939,en,King of New York,A former drug lord returns from prison determi...,...,0,0,0,0,0,0,0,1,0,1
2406,False,NaN,200000,"[{'id': 18, 'name': 'Drama'}, {'id': 27, 'name...",NaN,15856,tt0051744,en,House on Haunted Hill,Frederick Loren has invited five strangers to ...,...,0,0,0,0,0,0,0,0,0,1
605,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,47333,tt0116684,en,Jane Eyre,Jane Eyre is an orphan cast out as a young gir...,...,0,0,0,0,0,0,0,0,0,1
42402,False,NaN,0,"[{'id': 10770, 'name': 'TV Movie'}, {'id': 878...",NaN,160799,tt0070742,en,The Stranger,TV series pilot (series never launched). After...,...,0,0,0,0,0,0,0,0,0,1
3070,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,104878,tt0169333,en,Trans,"Ryan Kazinski, 16, is in juvenile detention in...",...,0,0,0,0,0,0,0,0,0,0


In [307]:
list(movies_data.columns)

['adult',
 'belongs_to_collection',
 'budget',
 'genres',
 'homepage',
 'id',
 'imdb_id',
 'original_language',
 'original_title',
 'overview',
 'popularity',
 'poster_path',
 'production_companies',
 'production_countries',
 'release_date',
 'revenue',
 'runtime',
 'spoken_languages',
 'status',
 'tagline',
 'title',
 'video',
 'vote_average',
 'vote_count',
 'release_year',
 'release_month',
 'release_day',
 'has_homepage',
 'is_in_series',
 'has_tagline',
 'has_poster',
 'has_overview',
 'is_animation',
 'is_comedy',
 'is_family',
 'is_adventure',
 'is_fantasy',
 'is_romance',
 'is_crime',
 'is_thriller',
 'is_mystery',
 'is_science_fiction',
 'is_music',
 'is_horror',
 'is_war',
 'is_documentary',
 'is_foreign',
 'is_tv_movie',
 'is_history',
 'is_drama',
 'is_western',
 'is_action',
 'Warner Bros.',
 'Metro-Goldwyn-Mayer (MGM)',
 'Paramount Pictures',
 'Twentieth Century Fox Film Corporation',
 'Universal Pictures',
 'Columbia Pictures Corporation',
 'Canal+',
 'Columbia Pictures'

The below columns are renamed in order to make them more pythonic and contain the appropriate information.

In [308]:
movies_data.rename(columns={'Warner Bros.': 'produced_by_warner_bros',
 'Metro-Goldwyn-Mayer (MGM)':'produced_by_mgm',
 'Paramount Pictures':'produced_by_paramount_pictures',
 'Twentieth Century Fox Film Corporation':'produced_by_20th_century_fox_film_corp',
 'Universal Pictures':'produced_by_universal_pictures',
 'Columbia Pictures Corporation':'produced_by_columbia_pictures_corp',
 'Canal+':'produced_by_canal',
 'Columbia Pictures':'produced_by_columbia_pictures',
 'RKO Radio Pictures':'produced_by_rko_radio_pictures',
 'New Line Cinema':'produced_by_new_line_cinema',
 'United Artists':'produced_by_united_aritists',
 'Walt Disney Pictures':'produced_by_walt_disney_pictures',
 'Touchstone Pictures':'produced_by_touchstone_pictures',
 'TriStar Pictures':'produced_by_tristar_pictures',
 'Mosfilm':'produced_by_mosfilm',
 'Miramax Films':'produced_by_miramax_films',
 'France 2 Cinéma':'produced_by_france_2_cinema',
 'Centre National de la Cinématographie (CNC)':'produced_by_cnc',
 'Toho Company':'produced_by_toho',
 'BBC Films':'produced_by_bbc_films',
 'Gaumont':'produced_by_gaumont',
 'StudioCanal':'produced_by_studiocanal',
 'Relativity Media':'produced_by_relativity_media',
 'British Broadcasting Corporation (BBC)':'produced_by_bbc_corp',
 'Walt Disney Productions':'produced_by_walt_disney_productions',
 'TF1 Films Production':'produced_by_tf1_films_productions',
 'Orion Pictures':'produced_by_orion_pictures',
 'Village Roadshow Pictures':'produced_by_village_roadshow_pictures',
 'Regency Enterprises':'produced_by_regency_enerprises',
 '普通话':'spoken_in_mandarin',
 'Français':'spoken_in_french',
 'Polski':'spoken_in_polish',
 'हिन्दी':'spoken_in_hindi',
 'Italiano':'spoken_in_italian',
 '日本語':'spoken_in_japanese',
 'English':'spoken_in_english',
 '한국어/조선말':'spoken_in_korean',
 '广州话 / 廣州話':'spoken_in_cantonese',
 'Pусский':'spoken_in_russian',
 'svenska':'spoken_in_swedish',
 'Português':'spoken_in_portuguese',
 'Deutsch':'spoken_in_dutch',
 'Español':'spoken_in_spanish',
 'Italy':'produced_in_italy',
 'France':'produced_in_france',
 'United States of America':'produced_in_usa',
 'Spain':'produced_in_spain',
 'India':'produced_in_india',
 'South Korea':'produced_in_south_korea',
 'Canada':'produced_in_canada',
 'Hong Kong':'produced_in_hong_kong',
 'Japan':'produced_in_japan',
 'Russia':'produced_in_russia',
 'Sweden':'produced_in_sweden',
 'Australia':'produced_in_australia',
 'Germany':'produced_in_germany',
 'United Kingdom':'produced_in_uk'},inplace=True)

In [309]:
list(movies_data.columns)

['adult',
 'belongs_to_collection',
 'budget',
 'genres',
 'homepage',
 'id',
 'imdb_id',
 'original_language',
 'original_title',
 'overview',
 'popularity',
 'poster_path',
 'production_companies',
 'production_countries',
 'release_date',
 'revenue',
 'runtime',
 'spoken_languages',
 'status',
 'tagline',
 'title',
 'video',
 'vote_average',
 'vote_count',
 'release_year',
 'release_month',
 'release_day',
 'has_homepage',
 'is_in_series',
 'has_tagline',
 'has_poster',
 'has_overview',
 'is_animation',
 'is_comedy',
 'is_family',
 'is_adventure',
 'is_fantasy',
 'is_romance',
 'is_crime',
 'is_thriller',
 'is_mystery',
 'is_science_fiction',
 'is_music',
 'is_horror',
 'is_war',
 'is_documentary',
 'is_foreign',
 'is_tv_movie',
 'is_history',
 'is_drama',
 'is_western',
 'is_action',
 'produced_by_warner_bros',
 'produced_by_mgm',
 'produced_by_paramount_pictures',
 'produced_by_20th_century_fox_film_corp',
 'produced_by_universal_pictures',
 'produced_by_columbia_pictures_corp',
 

The below information is used to determine the overview sentiment information and how long the overview is. This information is iterated through a list and can show the descriptive power of these overviews. THis information is then appended onto a single dataframe.

In [310]:
corpus_overview=list(set(movies_data['overview'][movies_data['has_overview']==1]))

In [311]:
corpus_overview[32:35]

['Italian comedy',
 'Shiro’s struggle with his father’s cancer and impending death leads to a realization that he must communicate his love and admiration for him before it’s too late. A series of flashbacks reveals their relationship over time, and the trouble Shiro faced connecting to his strict father who was also his teacher and soccer coach. With a consuming secret of his own, Shiro, now in his late twenties and about to get engaged, must eventually learn how to share the pain of it with his loved ones.',
 'God contacts Congressman Evan Baxter and tells him to build an ark in preparation for a great flood.']

In [312]:
all_scores_overview=[]
for i in corpus_overview:
    score=sia.polarity_scores(i)
    score['overview']=i
    all_scores_overview.append(score)
df_all_scores_overview=pd.DataFrame(all_scores_overview)

In [313]:
df_all_scores_overview.head()

,neg,neu,pos,compound,overview
0,0.233,0.743,0.025,-0.9118,"Members (Danny Dyer, Laura Harris, Tim McInner..."
1,0.044,0.878,0.078,0.2732,"Akira Saito, a Japanese businessman lives in T..."
2,0.283,0.680,0.037,-0.8658,"During a local fishing contest, people are bei..."
3,0.184,0.728,0.088,-0.6742,"England, 1932. Today is Dolly Thatcham's weddi..."
4,0.255,0.461,0.284,0.2716,A fragile Kansas girl's unrequited and forbidd...


In [314]:
df_all_scores_overview.rename(columns={
    'neg':'overview_negative_sentiment_score',
    'neu':'overview_neutral_sentiment_score',
    'pos':'overview_positive_sentiment_score',
    'compound':'overview_compound_sentiment_score'},inplace=True)
df_all_scores_overview.head()

,overview_negative_sentiment_score,overview_neutral_sentiment_score,overview_positive_sentiment_score,overview_compound_sentiment_score,overview
0,0.233,0.743,0.025,-0.9118,"Members (Danny Dyer, Laura Harris, Tim McInner..."
1,0.044,0.878,0.078,0.2732,"Akira Saito, a Japanese businessman lives in T..."
2,0.283,0.680,0.037,-0.8658,"During a local fishing contest, people are bei..."
3,0.184,0.728,0.088,-0.6742,"England, 1932. Today is Dolly Thatcham's weddi..."
4,0.255,0.461,0.284,0.2716,A fragile Kansas girl's unrequited and forbidd...


In [315]:
df_all_scores_overview['overview_length']=df_all_scores_overview.overview.map(len)
print(df_all_scores_overview.shape)
df_all_scores_overview.head()

(43823, 6)


,overview_negative_sentiment_score,overview_neutral_sentiment_score,overview_positive_sentiment_score,overview_compound_sentiment_score,overview,overview_length
0,0.233,0.743,0.025,-0.9118,"Members (Danny Dyer, Laura Harris, Tim McInner...",323
1,0.044,0.878,0.078,0.2732,"Akira Saito, a Japanese businessman lives in T...",352
2,0.283,0.680,0.037,-0.8658,"During a local fishing contest, people are bei...",185
3,0.184,0.728,0.088,-0.6742,"England, 1932. Today is Dolly Thatcham's weddi...",407
4,0.255,0.461,0.284,0.2716,A fragile Kansas girl's unrequited and forbidd...,153


In [316]:
movies_data=movies_data.merge(df_all_scores_overview,how='left',on='overview')
print(movies_data.shape)
movies_data.head()

(44686, 114)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,produced_in_india,produced_in_australia,produced_in_uk,produced_in_south_korea,produced_in_usa,overview_negative_sentiment_score,overview_neutral_sentiment_score,overview_positive_sentiment_score,overview_compound_sentiment_score,overview_length
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,0,0,0,0,1,0.048,0.886,0.066,0.2500,303.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,0,0,0,0,1,0.161,0.702,0.137,-0.4588,395.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,0,0,0,0,1,0.168,0.791,0.041,-0.7645,327.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,0,0,0,0,1,0.060,0.673,0.267,0.8860,270.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,0,0,0,0,1,0.000,0.944,0.056,0.5023,318.0


The below information is used to determine the tagline sentiment information and how long the tagline is. This information is iterated through a list and can show the descriptive power of these tagline. This information is then appended onto a single dataframe.

In [317]:
corpus_tagline=list(set(movies_data['tagline'][movies_data['has_tagline']==1]))
corpus_tagline[:10]

['Death is like a boomerang. it keeps coming back',
 'The Hidden Truth',
 'Human zombies rise from their coffins as living corpses!',
 'Nora Ephron: Scripted & Unscripted',
 'Right place. Wrong time.',
 "You can't exchange family.",
 'OUR VAGRANT YOUTH!',
 'There once was a wolf named Lupold...',
 '...a motion picture for all times!',
 'When life fell apart, love fell into place.']

In [318]:
all_scores_tagline=[]
for i in corpus_tagline:
    score=sia.polarity_scores(i)
    score['tagline']=i
    all_scores_tagline.append(score)
df_all_scores_tagline=pd.DataFrame(all_scores_tagline)
df_all_scores_tagline.head()

,neg,neu,pos,compound,tagline
0,0.315,0.484,0.202,-0.3400,Death is like a boomerang. it keeps coming back
1,0.000,0.465,0.535,0.3182,The Hidden Truth
2,0.000,1.000,0.000,0.0000,Human zombies rise from their coffins as livin...
3,0.000,1.000,0.000,0.0000,Nora Ephron: Scripted & Unscripted
4,0.508,0.492,0.000,-0.4767,Right place. Wrong time.


In [319]:
df_all_scores_tagline.rename(columns={
    'neg':'tagline_negative_sentiment_score',
    'neu':'tagline_neutral_sentiment_score',
    'pos':'tagline_positive_sentiment_score',
    'compound':'tagline_compound_sentiment_score'},inplace=True)

In [320]:
df_all_scores_tagline['tagline_length']=df_all_scores_tagline['tagline'].map(len)
df_all_scores_tagline.head(10)

,tagline_negative_sentiment_score,tagline_neutral_sentiment_score,tagline_positive_sentiment_score,tagline_compound_sentiment_score,tagline,tagline_length
0,0.315,0.484,0.202,-0.3400,Death is like a boomerang. it keeps coming back,47
1,0.000,0.465,0.535,0.3182,The Hidden Truth,16
2,0.000,1.000,0.000,0.0000,Human zombies rise from their coffins as livin...,56
3,0.000,1.000,0.000,0.0000,Nora Ephron: Scripted & Unscripted,34
4,0.508,0.492,0.000,-0.4767,Right place. Wrong time.,24
5,0.000,1.000,0.000,0.0000,You can't exchange family.,26
6,0.000,1.000,0.000,0.0000,OUR VAGRANT YOUTH!,18
7,0.000,1.000,0.000,0.0000,There once was a wolf named Lupold...,37
8,0.000,1.000,0.000,0.0000,...a motion picture for all times!,34
9,0.000,0.625,0.375,0.6369,"When life fell apart, love fell into place.",43


In [321]:
movies_data=movies_data.merge(df_all_scores_tagline,how='left',on='tagline')
print(movies_data.shape)
movies_data.head()

(44686, 119)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,overview_negative_sentiment_score,overview_neutral_sentiment_score,overview_positive_sentiment_score,overview_compound_sentiment_score,overview_length,tagline_negative_sentiment_score,tagline_neutral_sentiment_score,tagline_positive_sentiment_score,tagline_compound_sentiment_score,tagline_length
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,0.048,0.886,0.066,0.2500,303.0,NaN,NaN,NaN,NaN,NaN
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,0.161,0.702,0.137,-0.4588,395.0,0.000,0.632,0.368,0.5411,41.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,0.168,0.791,0.041,-0.7645,327.0,0.176,0.352,0.472,0.6369,52.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,0.060,0.673,0.267,0.8860,270.0,0.000,0.732,0.268,0.5812,78.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,0.000,0.944,0.056,0.5023,318.0,0.000,0.862,0.138,0.3382,78.0


The below information is used to determine the title and original title sentiment information and how long the title and original title is. This information is iterated through a list and can show the descriptive power of these titles. This information is then appended onto a single dataframe.

In [322]:
original_title_list=list(set(movies_data['original_title']))
original_title_list[:10]

['The Initiation of Sarah',
 'The Art of War II: Betrayal',
 'Holiday Inn',
 '2046',
 'Rockaway',
 'Me and the Colonel',
 'Deathgasm',
 'Tiptoes',
 'Clonehunter',
 'Rage in Heaven']

In [323]:
title_list=list(set(movies_data['title']))
title_list[:10]

['Love Without Pity',
 'A Crime in Paradise',
 'The Initiation of Sarah',
 'Manhunt',
 'The Art of War II: Betrayal',
 'Holiday Inn',
 '2046',
 'Rockaway',
 'Me and the Colonel',
 'Drunken Master Part II - Dance of the Drunk Mantis']

In [324]:
all_scores_title_list=[]
for i in title_list:
    score=sia.polarity_scores(i)
    score['title']=i
    all_scores_title_list.append(score)
df_all_scores_title_list=pd.DataFrame(all_scores_title_list)
df_all_scores_title_list.head()

,neg,neu,pos,compound,title
0,0.000,0.141,0.859,0.7259,Love Without Pity
1,0.402,0.115,0.483,0.1779,A Crime in Paradise
2,0.000,1.000,0.000,0.0000,The Initiation of Sarah
3,0.000,1.000,0.000,0.0000,Manhunt
4,0.658,0.342,0.000,-0.8271,The Art of War II: Betrayal


In [325]:
df_all_scores_title_list.rename(columns={'neg':'title_negative_sentiment_score','neu':'title_neutral_sentiment_score','pos':'title_postive_sentiment_score','compound':'title_compound_sentiment_score'},inplace=True)
df_all_scores_title_list.head()

,title_negative_sentiment_score,title_neutral_sentiment_score,title_postive_sentiment_score,title_compound_sentiment_score,title
0,0.000,0.141,0.859,0.7259,Love Without Pity
1,0.402,0.115,0.483,0.1779,A Crime in Paradise
2,0.000,1.000,0.000,0.0000,The Initiation of Sarah
3,0.000,1.000,0.000,0.0000,Manhunt
4,0.658,0.342,0.000,-0.8271,The Art of War II: Betrayal


In [326]:
all_scores_original_title_list=[]
for i in original_title_list:
    score=sia.polarity_scores(i)
    score['original_title']=i
    all_scores_original_title_list.append(score)
df_all_scores_original_title_list=pd.DataFrame(all_scores_original_title_list)
df_all_scores_original_title_list.head()

,neg,neu,pos,compound,original_title
0,0.000,1.000,0.00,0.0000,The Initiation of Sarah
1,0.658,0.342,0.00,-0.8271,The Art of War II: Betrayal
2,0.000,0.270,0.73,0.4019,Holiday Inn
3,0.000,1.000,0.00,0.0000,2046
4,0.000,1.000,0.00,0.0000,Rockaway


In [327]:
df_all_scores_original_title_list.rename(columns={'neg':'original_title_negative_sentiment_score','neu':'original_title_neutral_sentiment_score','pos':'original_title_postive_sentiment_score','compound':'original_title_compound_sentiment_score'},inplace=True)
df_all_scores_original_title_list.head()

,original_title_negative_sentiment_score,original_title_neutral_sentiment_score,original_title_postive_sentiment_score,original_title_compound_sentiment_score,original_title
0,0.000,1.000,0.00,0.0000,The Initiation of Sarah
1,0.658,0.342,0.00,-0.8271,The Art of War II: Betrayal
2,0.000,0.270,0.73,0.4019,Holiday Inn
3,0.000,1.000,0.00,0.0000,2046
4,0.000,1.000,0.00,0.0000,Rockaway


In [328]:
movies_data['original_title_length']=movies_data['original_title'].map(len)

In [329]:
movies_data['title_length']=movies_data.title.map(len)

In [330]:
movies_data=movies_data.merge(df_all_scores_title_list,how='left',on='title')
movies_data=movies_data.merge(df_all_scores_original_title_list,how='left',on='original_title')
movies_data.head(10)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,original_title_length,title_length,title_negative_sentiment_score,title_neutral_sentiment_score,title_postive_sentiment_score,title_compound_sentiment_score,original_title_negative_sentiment_score,original_title_neutral_sentiment_score,original_title_postive_sentiment_score,original_title_compound_sentiment_score
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,9,9,0.000,1.000,0.0,0.0000,0.000,1.000,0.0,0.0000
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,7,7,0.000,1.000,0.0,0.0000,0.000,1.000,0.0,0.0000
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,16,16,0.000,1.000,0.0,0.0000,0.000,1.000,0.0,0.0000
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,17,17,0.000,1.000,0.0,0.0000,0.000,1.000,0.0,0.0000
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,27,27,0.000,1.000,0.0,0.0000,0.000,1.000,0.0,0.0000
5,False,NaN,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,4,4,0.000,1.000,0.0,0.0000,0.000,1.000,0.0,0.0000
6,False,NaN,58000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,11860,tt0114319,en,Sabrina,An ugly duckling having undergone a remarkable...,...,7,7,0.000,1.000,0.0,0.0000,0.000,1.000,0.0,0.0000
7,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,45325,tt0112302,en,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses...",...,12,12,0.000,1.000,0.0,0.0000,0.000,1.000,0.0,0.0000
8,False,NaN,35000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,9091,tt0114576,en,Sudden Death,International action superstar Jean Claude Van...,...,12,12,0.796,0.204,0.0,-0.5994,0.796,0.204,0.0,-0.5994
9,False,"{'id': 645, 'name': 'James Bond Collection', '...",58000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,en,GoldenEye,James Bond must unmask the mysterious head of ...,...,9,9,0.000,1.000,0.0,0.0000,0.000,1.000,0.0,0.0000


In [331]:
list(movies_data.columns)

['adult',
 'belongs_to_collection',
 'budget',
 'genres',
 'homepage',
 'id',
 'imdb_id',
 'original_language',
 'original_title',
 'overview',
 'popularity',
 'poster_path',
 'production_companies',
 'production_countries',
 'release_date',
 'revenue',
 'runtime',
 'spoken_languages',
 'status',
 'tagline',
 'title',
 'video',
 'vote_average',
 'vote_count',
 'release_year',
 'release_month',
 'release_day',
 'has_homepage',
 'is_in_series',
 'has_tagline',
 'has_poster',
 'has_overview',
 'is_animation',
 'is_comedy',
 'is_family',
 'is_adventure',
 'is_fantasy',
 'is_romance',
 'is_crime',
 'is_thriller',
 'is_mystery',
 'is_science_fiction',
 'is_music',
 'is_horror',
 'is_war',
 'is_documentary',
 'is_foreign',
 'is_tv_movie',
 'is_history',
 'is_drama',
 'is_western',
 'is_action',
 'produced_by_warner_bros',
 'produced_by_mgm',
 'produced_by_paramount_pictures',
 'produced_by_20th_century_fox_film_corp',
 'produced_by_universal_pictures',
 'produced_by_columbia_pictures_corp',
 

In [332]:
movies_data['title_change']=movies_data['original_title']!=movies_data['title']

In [333]:
movies_data['title_change']=movies_data['title_change'].astype(int)

In [334]:
movies_data['title_change'].value_counts(normalize=True)

0    0.747751
1    0.252249
Name: title_change, dtype: float64

The keywords, casting, key and ratings information is then merged into the movie data data frame. This will allow all of the information to be stored into a single dataframe for mean imputation and modeling.

In [335]:
movies_data['id']=movies_data['id'].astype(int)

In [336]:
movies_data=movies_data.merge(key_words_to_manipulate,how='left',left_on='id', right_on='movie_id')
movies_data.sample(10)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,movie_id,total_movie_keywords,most_used_keywords_per_movie,second_most_used_keywords_per_movie,second_least_used_keywords_per_movie,least_used_keywords_per_movie,average_keyword_negative_sentiment_per_movie,average_keyword_neutral_sentiment_per_movie,average_keyword_postive_sentiment_per_movie,average_keyword_compound_sentiment_per_movie
41233,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}]",NaN,122134,tt0000029,fr,Repas de Bébé,"A father, a mother and a baby are sitting at a...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42764,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}]",NaN,413782,tt4000670,en,Mifune: The Last Samurai,Nearly 20 years after his death and in the run...,...,413782.0,3.0,0.0,0.0,0.0,267.0,0.000000,1.000000,0.000000,0.000000
18086,False,NaN,500000,"[{'id': 18, 'name': 'Drama'}]",NaN,66488,tt0065086,en,That Cold Day in the Park,"A rich but lonely woman, Frances Austen, one d...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27615,False,NaN,1131974,"[{'id': 9648, 'name': 'Mystery'}, {'id': 80, '...",NaN,318007,tt2071582,no,Skumringslandet,The investigation of a mysterious serial-kille...,...,318007.0,2.0,0.0,0.0,293.0,18.0,0.764061,0.235939,0.000000,-0.611060
33006,False,"{'id': 164072, 'name': 'Baby Burlesks', 'poste...",0,[],NaN,156388,tt0023337,en,The Pie-Covered Wagon,Baby Burlesk on wagon train fight with Indians.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10958,False,NaN,1000000,"[{'id': 99, 'name': 'Documentary'}]",http://www.whokilledtheelectriccar.com/,13508,tt0489037,en,Who Killed the Electric Car?,"In 1996, electric cars began to appear on road...",...,13508.0,3.0,0.0,1930.0,0.0,8.0,0.000000,0.998952,0.001048,0.000423
19265,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 35, 'nam...",NaN,16640,tt0477065,en,Pirates of the Great Salt Lake,Kirk Redgrave and Flint Weaver met only three ...,...,16640.0,2.0,0.0,1930.0,0.0,5.0,0.000000,1.000000,0.000000,0.000000
23430,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,120673,tt0015498,en,Wild Oranges,Millie Stope lives with her grandfather on a r...,...,120673.0,8.0,0.0,0.0,306.0,364.0,0.408955,0.591045,0.000000,-0.122292
40964,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,228384,tt2633110,de,Eltern,Christine and Conrad are what we call modern p...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31464,False,NaN,0,"[{'id': 27, 'name': 'Horror'}, {'id': 35, 'nam...",NaN,144351,tt0093088,en,Ghost Fever,Two policeman are sent on a routine assignment...,...,144351.0,1.0,0.0,0.0,0.0,6.0,0.000000,1.000000,0.000000,0.000000


In [337]:
movies_data=movies_data.merge(crews, how='left', left_on='id',right_on='movie_id')
movies_data.sample(10)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,average_keyword_negative_sentiment_per_movie,average_keyword_neutral_sentiment_per_movie,average_keyword_postive_sentiment_per_movie,average_keyword_compound_sentiment_per_movie,movie_id_y,total_movie_crew,most_used_crew_member_per_movie,second_most_used_crew_member_per_movie,second_least_used_crew_member_per_movie,least_used_crew_member_per_movie
203672,False,NaN,0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://www.dasrad.com,37060,tt0330801,de,Das Rad,The stone-people Hew and Kew have seen a lot i...,...,0.000000,1.000000,0.000000,0.000000,37060.0,7.0,0.0,0.0,6.0,1.0
426895,False,"{'id': 414000, 'name': 'Omae Umasou Da na', 'p...",0,"[{'id': 16, 'name': 'Animation'}, {'id': 12, '...",http://www.poplar.co.jp/special/umasou/,89825,tt1734113,ja,おまえうまそうだな,You Are Umasou adapts Tatsuya Miyanishi's popu...,...,0.000000,1.000000,0.000000,0.000000,89825.0,9.0,0.0,0.0,6.0,3.0
321629,False,NaN,900000,"[{'id': 18, 'name': 'Drama'}, {'id': 27, 'name...",NaN,71945,tt0067491,en,Night of Dark Shadows,A painter and his wife move into a home and fi...,...,0.909028,0.090972,0.000000,-0.388864,71945.0,4.0,2.0,0.0,2.0,0.0
276323,False,NaN,12000000,"[{'id': 18, 'name': 'Drama'}, {'id': 12, 'name...",NaN,129670,tt1821549,en,Nebraska,"An aging, booze-addled father takes a trip fro...",...,0.002065,0.997935,0.000000,-0.000712,129670.0,69.0,12.0,27.0,18.0,12.0
48326,False,NaN,100000000,"[{'id': 28, 'name': 'Action'}, {'id': 14, 'nam...",http://www.end-of-days.com/,9946,tt0146675,en,End of Days,"On December 28th, 1999, the citizens of New Yo...",...,0.171390,0.811206,0.017404,-0.100345,9946.0,139.0,36.0,53.0,30.0,20.0
1953,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 53, 'name...",NaN,27526,tt0112744,en,The Crossing Guard,"After his daughter died in a hit and run, Fred...",...,0.212651,0.787349,0.000000,-0.114278,27526.0,11.0,9.0,0.0,2.0,0.0
219430,False,NaN,38000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,38408,tt1055292,en,Life As We Know It,After a distastrous first date for caterer Hol...,...,0.037941,0.621916,0.340064,0.180123,38408.0,83.0,25.0,24.0,22.0,12.0
37969,False,NaN,50000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,10312,tt0129290,en,Patch Adams,"Meet Patch Adams, a doctor who doesn't look, a...",...,0.000000,0.994152,0.005848,0.002888,10312.0,54.0,19.0,21.0,8.0,6.0
241169,False,NaN,0,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",NaN,25922,tt0984155,en,3 días (Before the Fall),A meteorite will destroy the world in three da...,...,NaN,NaN,NaN,NaN,25922.0,19.0,0.0,2.0,12.0,5.0
238540,False,NaN,0,"[{'id': 9648, 'name': 'Mystery'}, {'id': 18, '...",NaN,48778,tt0343524,fr,Ce jour-là,A father (Michel Piccoli) is scheming to have ...,...,NaN,NaN,NaN,NaN,48778.0,5.0,3.0,2.0,0.0,0.0


In [338]:
movies_data=movies_data.merge(castings, how='left', left_on='id',right_on='movie_id')
movies_data.sample(10)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,most_used_crew_member_per_movie,second_most_used_crew_member_per_movie,second_least_used_crew_member_per_movie,least_used_crew_member_per_movie,movie_id,total_cast_per_movie,most_used_actor_per_movie,second_most_used_actor_per_movie,second_least_used_actor_per_movie,least_used_actor_per_movie
391087,False,NaN,0,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",NaN,256959,tt4287610,pt,"Branco Sai, Preto Fica",Shots fired inside a club frequented by black ...,...,0.0,0.0,8.0,5.0,256959.0,5.0,0.0,0.0,0.0,5.0
308267,False,NaN,0,"[{'id': 80, 'name': 'Crime'}, {'id': 99, 'name...",NaN,289575,tt3904186,en,Tales of the Grim Sleeper,When Lonnie Franklin Jr. was arrested in South...,...,2.0,4.0,1.0,3.0,289575.0,1.0,0.0,0.0,0.0,1.0
128678,False,NaN,0,"[{'id': 12, 'name': 'Adventure'}, {'id': 27, '...",NaN,3031,tt0024188,en,Island of Lost Souls,An obsessed scientist conducts profane experim...,...,5.0,4.0,1.0,0.0,3031.0,12.0,7.0,4.0,0.0,1.0
270835,False,"{'id': 363369, 'name': 'Pacific Rim Collection...",180000000,"[{'id': 28, 'name': 'Action'}, {'id': 878, 'na...",http://www.pacificrimmovie.com/,68726,tt1663662,en,Pacific Rim,"When legions of monstrous creatures, known as ...",...,15.0,8.0,4.0,0.0,68726.0,43.0,7.0,15.0,14.0,7.0
414731,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 53, 'name...",http://septembersofshiraz.film/,352179,tt3661298,en,Septembers of Shiraz,In this adaptation of the critically acclaimed...,...,7.0,23.0,19.0,14.0,352179.0,26.0,3.0,8.0,7.0,8.0
436914,False,NaN,1500000,"[{'id': 18, 'name': 'Drama'}]",http://imnotashamedfilm.com/,409502,tt4950110,en,I'm Not Ashamed,Based on the inspiring and powerful true story...,...,0.0,1.0,1.0,8.0,409502.0,16.0,0.0,1.0,7.0,8.0
191756,False,NaN,0,"[{'id': 80, 'name': 'Crime'}, {'id': 9648, 'na...",http://www.alphabetkiller.com/,13942,tt0818165,en,The Alphabet Killer,Based on the true story of double killings occ...,...,3.0,11.0,5.0,4.0,13942.0,5.0,4.0,0.0,1.0,0.0
343465,False,NaN,11000000,"[{'id': 18, 'name': 'Drama'}]",http://womaningoldmovie.com/,304357,tt2404425,en,Woman in Gold,"Maria Altmann, an octogenarian Jewish refugee,...",...,11.0,16.0,3.0,1.0,304357.0,33.0,14.0,5.0,7.0,7.0
376644,False,NaN,0,"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name...",NaN,32577,tt0283534,en,The Parole Officer,A hapless parole officer is framed for murder ...,...,6.0,7.0,2.0,5.0,32577.0,33.0,9.0,6.0,10.0,8.0
435423,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",https://www.facebook.com/QueCulpaTieneElNino/,369299,tt5538800,es,¿Qué culpa tiene el niño?,"Maru, after becoming pregnant from a drunken o...",...,0.0,3.0,3.0,2.0,369299.0,14.0,0.0,3.0,4.0,7.0


In [339]:
movies_data=movies_data.merge(max_ratings_per_movie,how='left',left_on='id',right_on='movieId')
movies_data.sample(10)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,second_least_used_crew_member_per_movie,least_used_crew_member_per_movie,movie_id,total_cast_per_movie,most_used_actor_per_movie,second_most_used_actor_per_movie,second_least_used_actor_per_movie,least_used_actor_per_movie,movieId,max_rating
157618,False,NaN,30000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",http://www.sonypictures.com/movies/ultraviolet/,9920,tt0370032,en,Ultraviolet,"In the late 21st century, a subculture of huma...",...,18.0,24.0,9920.0,21.0,3.0,3.0,6.0,9.0,NaN,NaN
248509,False,NaN,4500000,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",http://www.notrejourviendra.com/,50365,tt1651328,fr,Notre jour viendra,Redheaded teen Remy is bullied by his soccer t...,...,21.0,41.0,50365.0,43.0,2.0,3.0,9.0,29.0,NaN,NaN
457319,False,NaN,100000000,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",http://www.dunkirkmovie.com/,374720,tt5013056,en,Dunkirk,The miraculous evacuation of Allied soldiers f...,...,59.0,84.0,374720.0,66.0,5.0,9.0,25.0,27.0,NaN,NaN
265532,False,"{'id': 135468, 'name': 'G.I. Joe (Live-Action)...",130000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",http://www.gijoemovie.com,72559,tt1583421,en,G.I. Joe: Retaliation,"Framed for crimes against the country, the G.I...",...,0.0,0.0,72559.0,20.0,10.0,6.0,4.0,0.0,72559.0,4.5
252542,False,NaN,6000000,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,122924,tt1881024,en,One in the Chamber,A seasoned assassin plays both sides in a Russ...,...,11.0,1.0,122924.0,6.0,3.0,2.0,1.0,0.0,122924.0,5.0
190175,False,NaN,0,"[{'id': 9648, 'name': 'Mystery'}, {'id': 53, '...",NaN,12449,tt0990361,fr,La Chambre des morts,While on a joyride with the headlights turned ...,...,3.0,1.0,12449.0,15.0,3.0,6.0,6.0,0.0,NaN,NaN
34930,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",http://www.imdb.com/title/tt0100134/,91217,tt0100134,en,Men Don't Leave,"After her husband John dies unexpectedly, Beth...",...,0.0,0.0,91217.0,12.0,7.0,4.0,1.0,0.0,NaN,NaN
23922,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,407992,tt0116949,en,MURDER and murder,Mildred and Doris are two middle-aged white wo...,...,1.0,8.0,407992.0,10.0,0.0,4.0,1.0,5.0,NaN,NaN
7072,False,NaN,57000000,"[{'id': 80, 'name': 'Crime'}, {'id': 28, 'name...",http://www.warnerbros.com/demolition-man,9739,tt0106697,en,Demolition Man,"Simon Phoenix, a violent criminal cryogenicall...",...,39.0,12.0,9739.0,22.0,11.0,9.0,0.0,2.0,NaN,NaN
344619,False,NaN,0,"[{'id': 27, 'name': 'Horror'}, {'id': 878, 'na...",NaN,3164,tt0051630,en,Frankenstein 1970,The baron's grandson rents the family castle t...,...,1.0,2.0,3164.0,12.0,2.0,4.0,5.0,1.0,3164.0,4.0


In [340]:
movies_data=movies_data.merge(min_ratings_per_movie,how='left',left_on='id',right_on='movieId')
movies_data.sample(10)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,movie_id,total_cast_per_movie,most_used_actor_per_movie,second_most_used_actor_per_movie,second_least_used_actor_per_movie,least_used_actor_per_movie,movieId_x,max_rating,movieId_y,min_rating
142066,False,NaN,43000000,"[{'id': 28, 'name': 'Action'}, {'id': 14, 'nam...",http://marvel.com/movies/movie/13/elektra,9947,tt0357277,en,Elektra,Elektra the warrior survives a near-death expe...,...,9947.0,20.0,6.0,9.0,3.0,2.0,NaN,NaN,NaN,NaN
346989,False,"{'id': 52985, 'name': 'Wrong Turn Collection',...",1200000,"[{'id': 27, 'name': 'Horror'}]",NaN,259072,tt3612032,en,Wrong Turn 6: Last Resort,A sudden and mysterious inheritance brings Dan...,...,259072.0,18.0,0.0,0.0,10.0,8.0,NaN,NaN,NaN,NaN
198116,False,NaN,12000000,"[{'id': 18, 'name': 'Drama'}, {'id': 14, 'name...",NaN,8884,tt0893402,en,Franklyn,Set between the parallel worlds of contemporar...,...,8884.0,14.0,3.0,8.0,1.0,2.0,8884.0,5.0,8884.0,0.5
221873,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,13834,tt0384699,en,Missing in America,"A haunted Vietnam veteran, living in exile in ...",...,13834.0,11.0,6.0,4.0,1.0,0.0,NaN,NaN,NaN,NaN
165454,False,NaN,40000000,"[{'id': 18, 'name': 'Drama'}, {'id': 9648, 'na...",http://wwws.warnerbros.de/theprestige/,1124,tt0482571,en,The Prestige,A mysterious story of two magicians whose inte...,...,1124.0,41.0,12.0,9.0,10.0,10.0,1124.0,5.0,1124.0,0.5
252193,False,"{'id': 425164, 'name': 'Dragon Ball Z (Movie) ...",0,"[{'id': 28, 'name': 'Action'}, {'id': 16, 'nam...",NaN,39107,tt0142236,ja,Doragon Bōru Zetto: Fukkatsu no Fyūjon!! Gokū ...,"Not paying attention to his job, a young demon...",...,39107.0,15.0,10.0,5.0,0.0,0.0,NaN,NaN,NaN,NaN
450776,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,127803,tt1718998,en,3 Times a Charm,NaN,...,127803.0,10.0,0.0,0.0,6.0,4.0,NaN,NaN,NaN,NaN
426251,False,NaN,2000000,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,311146,tt4235342,id,Pendekar Tongkat Emas,"Cempaka, a respected warrior and highly respec...",...,311146.0,8.0,0.0,0.0,6.0,2.0,NaN,NaN,NaN,NaN
417004,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 28, 'nam...",http://ourrobocopremake.com/,254251,tt3528906,en,Our RoboCop Remake,Our RoboCop Remake is a crowd-sourced film pro...,...,254251.0,6.0,1.0,0.0,2.0,3.0,NaN,NaN,NaN,NaN
302328,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",http://www.thewarboys.com/,39958,tt1070887,en,The War Boys,Three young vigilantes huddle on la linea read...,...,39958.0,7.0,1.0,3.0,2.0,1.0,NaN,NaN,NaN,NaN


In [341]:
movies_data=movies_data.merge(median_ratings,how='left',left_on='id',right_on='movieId')
movies_data.sample(10)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,most_used_actor_per_movie,second_most_used_actor_per_movie,second_least_used_actor_per_movie,least_used_actor_per_movie,movieId_x,max_rating,movieId_y,min_rating,movieId,median_rating
163845,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,15097,tt0109440,en,La cité de la peur,A second-class horror movie has to be shown at...,...,6.0,11.0,13.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN
194317,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,99283,tt0022279,en,Private Lives,Elyot and Sibyl are being married in a big chu...,...,7.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
348559,False,NaN,7500000,"[{'id': 28, 'name': 'Action'}]",NaN,326284,tt3488328,en,Pound of Flesh,In China to donate his kidney to his dying nie...,...,1.0,3.0,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
277505,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}]",http://www.fisheadmovie.com,99279,tt2856738,en,I am Fishead,A provocative snapshot of the world we live in...,...,1.0,0.0,4.0,6.0,99279.0,4.5,99279.0,3.5,99279.0,4.0
194668,False,"{'id': 9485, 'name': 'The Fast and the Furious...",85000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",http://www.fastandfuriousmovie.net,13804,tt1013752,en,Fast & Furious,"When a crime brings them back to L.A., fugitiv...",...,9.0,17.0,13.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN
205351,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}]",NaN,46886,tt1421032,en,Disko ja tuumasõda,A story about growing up in the Soviet Union. ...,...,1.0,0.0,1.0,13.0,NaN,NaN,NaN,NaN,NaN,NaN
414416,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10769, 'n...",http://mirchthemovie.com/,54142,tt1388903,en,Mirch,Maanav is a struggling filmmaker who will not ...,...,1.0,4.0,3.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
300778,False,NaN,16000000,"[{'id': 37, 'name': 'Western'}, {'id': 18, 'na...",NaN,264656,tt2398231,en,The Homesman,When three women living on the edge of the Ame...,...,11.0,6.0,4.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
150354,False,"{'id': 332357, 'name': 'The Dukes Of Hazzard C...",50000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://www.warnerbros.com/dukes-hazzard/index_...,6519,tt0377818,en,The Dukes of Hazzard,"Cousins, Bo and Luke Duke, with the help of th...",...,8.0,6.0,6.0,2.0,6519.0,5.0,6519.0,0.5,6519.0,3.0
213543,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}]",http://www.sonypictures.de/landing/friendship/,35217,tt1247657,de,Friendship!,A young man from East Germany travels to San F...,...,2.0,4.0,3.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [342]:
movies_data=movies_data.merge(average_ratings,how='left',left_on='id',right_on='movieId')
movies_data.head(10)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,second_least_used_actor_per_movie,least_used_actor_per_movie,movieId_x,max_rating,movieId_y,min_rating,movieId_x,median_rating,movieId_y,average_rating
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,2.0,0.0,862.0,5.0,862.0,1.0,862.0,4.0,862.0,3.59893
1,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,2.0,0.0,862.0,5.0,862.0,1.0,862.0,4.0,862.0,3.59893
2,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,2.0,0.0,862.0,5.0,862.0,1.0,862.0,4.0,862.0,3.59893
3,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,2.0,0.0,862.0,5.0,862.0,1.0,862.0,4.0,862.0,3.59893
4,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,2.0,0.0,862.0,5.0,862.0,1.0,862.0,4.0,862.0,3.59893
5,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,2.0,0.0,862.0,5.0,862.0,1.0,862.0,4.0,862.0,3.59893
6,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,2.0,0.0,862.0,5.0,862.0,1.0,862.0,4.0,862.0,3.59893
7,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,2.0,0.0,862.0,5.0,862.0,1.0,862.0,4.0,862.0,3.59893
8,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,2.0,0.0,862.0,5.0,862.0,1.0,862.0,4.0,862.0,3.59893
9,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,2.0,0.0,862.0,5.0,862.0,1.0,862.0,4.0,862.0,3.59893


In [343]:
movies_data=movies_data.merge(mode_ratings,how='left',left_on='id',right_on='movieId')
movies_data.head(10)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,movieId_x,max_rating,movieId_y,min_rating,movieId_x,median_rating,movieId_y,average_rating,movieId,mode_rating
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,862.0,5.0,862.0,1.0,862.0,4.0,862.0,3.59893,862.0,4.0
1,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,862.0,5.0,862.0,1.0,862.0,4.0,862.0,3.59893,862.0,4.0
2,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,862.0,5.0,862.0,1.0,862.0,4.0,862.0,3.59893,862.0,4.0
3,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,862.0,5.0,862.0,1.0,862.0,4.0,862.0,3.59893,862.0,4.0
4,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,862.0,5.0,862.0,1.0,862.0,4.0,862.0,3.59893,862.0,4.0
5,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,862.0,5.0,862.0,1.0,862.0,4.0,862.0,3.59893,862.0,4.0
6,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,862.0,5.0,862.0,1.0,862.0,4.0,862.0,3.59893,862.0,4.0
7,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,862.0,5.0,862.0,1.0,862.0,4.0,862.0,3.59893,862.0,4.0
8,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,862.0,5.0,862.0,1.0,862.0,4.0,862.0,3.59893,862.0,4.0
9,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,862.0,5.0,862.0,1.0,862.0,4.0,862.0,3.59893,862.0,4.0


In [344]:
list(movies_data.columns)

['adult',
 'belongs_to_collection',
 'budget',
 'genres',
 'homepage',
 'id',
 'imdb_id',
 'original_language',
 'original_title',
 'overview',
 'popularity',
 'poster_path',
 'production_companies',
 'production_countries',
 'release_date',
 'revenue',
 'runtime',
 'spoken_languages',
 'status',
 'tagline',
 'title',
 'video',
 'vote_average',
 'vote_count',
 'release_year',
 'release_month',
 'release_day',
 'has_homepage',
 'is_in_series',
 'has_tagline',
 'has_poster',
 'has_overview',
 'is_animation',
 'is_comedy',
 'is_family',
 'is_adventure',
 'is_fantasy',
 'is_romance',
 'is_crime',
 'is_thriller',
 'is_mystery',
 'is_science_fiction',
 'is_music',
 'is_horror',
 'is_war',
 'is_documentary',
 'is_foreign',
 'is_tv_movie',
 'is_history',
 'is_drama',
 'is_western',
 'is_action',
 'produced_by_warner_bros',
 'produced_by_mgm',
 'produced_by_paramount_pictures',
 'produced_by_20th_century_fox_film_corp',
 'produced_by_universal_pictures',
 'produced_by_columbia_pictures_corp',
 

All unnecessary columns are dropped and the dataframe is exported for mean imputation.

In [345]:
movies_data.drop(columns=['imdb_id',
 'original_language',
 'original_title',
 'overview',
 'poster_path',
 'production_companies',
 'production_countries',
 'release_date',
 'adult',
 'belongs_to_collection',
 'budget',
 'genres',
 'homepage',
 'spoken_languages',
 'status',
 'tagline',
 'title',
 'movie_id',
 'movieId_y',
 'movieId_x',
 'movieId',
'revenue'],inplace=True)
print(movies_data.shape)
movies_data.head()

(462297, 138)


,id,popularity,runtime,video,vote_average,vote_count,release_year,release_month,release_day,has_homepage,...,total_cast_per_movie,most_used_actor_per_movie,second_most_used_actor_per_movie,second_least_used_actor_per_movie,least_used_actor_per_movie,max_rating,min_rating,median_rating,average_rating,mode_rating
0,862,21.9469,81.0,0,7.7,5415.0,1995,10,30,1,...,13.0,9.0,2.0,2.0,0.0,5.0,1.0,4.0,3.59893,4.0
1,862,21.9469,81.0,0,7.7,5415.0,1995,10,30,1,...,13.0,9.0,2.0,2.0,0.0,5.0,1.0,4.0,3.59893,4.0
2,862,21.9469,81.0,0,7.7,5415.0,1995,10,30,1,...,13.0,9.0,2.0,2.0,0.0,5.0,1.0,4.0,3.59893,4.0
3,862,21.9469,81.0,0,7.7,5415.0,1995,10,30,1,...,13.0,9.0,2.0,2.0,0.0,5.0,1.0,4.0,3.59893,4.0
4,862,21.9469,81.0,0,7.7,5415.0,1995,10,30,1,...,13.0,9.0,2.0,2.0,0.0,5.0,1.0,4.0,3.59893,4.0


In [348]:
movies_data.drop_duplicates(inplace=True)
print(movies_data.shape)

(44674, 138)


In [349]:
movies_data.to_csv('../Data/movies_data_complete.csv')